<a href="https://colab.research.google.com/github/EduGame3/Experiment/blob/master/Tarea2_ADRF_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 447 (delta 105), reused 0 (delta 0), pack-reused 311
Receiving objects: 100% (447/447), 1.53 MiB | 17.41 MiB/s, done.
Resolving deltas: 100% (288/288), done.


In [0]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
#require(data.table)
install.packages("data.table")
library(data.table)
#require("PerformanceAnalytics")
install.packages("PerformanceAnalytics")
library("PerformanceAnalytics")

install.packages("Deriv")
library(Deriv)


library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/quantmod_0.4.17.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 152204 bytes (148 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

In [0]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}

#interpolación
talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


# Definición de parámetros para valorar:

Cargamos parámetros para valoración:

In [0]:
%%R
fval=as.Date("20200228",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.95 #Nivel de confianza para obtener estimaciones de riesgo
#setwd(direc)	

#ACCIONES Y DIVISAS
#Cargar los símbolos de yahoo finance para EQ
Symbols<-c  ("AMXL.MX","GCARSOA1.MX", "WMT.MX" )#tienen que ir en orden alfabético
pos_eq=c(-5000,1000,1200) #monto inicial invertido en acciones
#Cargar los símbolos de yahoo finance para FX
SymbolsFX<-c("EURUSD=X","GBPUSD=X","USDMXN=X" ) #tienen que ir en orden alfabético
pos_fx=c(700,-600,1500) #monto inicial invertido en divisas
nh=3660 #días de historia

#CETES
base="RiesgosFinancieros/2020-2/Tarea/tasa_guber.txt" #curva gubernamental
plazos_bcc=cbind(180) #plazo del cupón
contratos_bcc=cbind(1500) #número de contratos
nominal_bcc=10 #CETE

#BONOS M
btasadesc_bm="RiesgosFinancieros/2020-2/Tarea/tasa_yield.txt" 
tfcupon_bm=cbind(0.065) #Tasafija del cupón
plazos_bm=cbind(3600) #Vencimiento del bono
plazocupon_bm=cbind(360) #plazos_bm fijos de cada cupón
contratos_bm=cbind(-10000) #posición invertida
nominal_bm=100

#BONDES D
btasadescst="RiesgosFinancieros/2020-2/Tarea/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2020-2/Tarea/tfondeo.txt"
plazos_bdm=cbind(707) #Vencimiento del bono
plazocupon_bdm=cbind(28) #plazos_bdm fijos de cada cupón
contratos_bdm=cbind(1000) #posición invertida
nominal_bdm=100

#FORWARDS TDC
bext="RiesgosFinancieros/2020-2/Tarea/tasa_libor.txt"
bdom="RiesgosFinancieros/2020-2/Tarea/tasa_fwd.txt"
btsp="RiesgosFinancieros/2020-2/Tarea/tasa_spot.txt"
SymbolsFX_ftdc<-c("USDMXN=X", "GBPUSD=X" ) #tienen que ir en orden alfabético
plazos_fwd=cbind(5)
contratos_fwd=cbind(100)
kst_fwd=cbind(19.83)
nominal_fwd=1
yext=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

#FORWARDS DE IPC
#Descontamos con gubernamental
SymbolsEQ_find<-c("^MXX", "GCARSOA1.MX" ) #tienen que ir en orden alfabético
plazos_fwd_ind=cbind(53)
contratos_fwd_ind=cbind(50)
kst_fwd_ind=cbind(-49525)
nominal_fwd_ind=1

# SWAPS

btasadesc_sw="RiesgosFinancieros/2020-2/Tarea/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="RiesgosFinancieros/2020-2/Tarea/tasa_DIRS_SW_OP.txt"
tasafija_sw=cbind(0.052,0.049) #se establece la tasa fija a pagar para cada swap
plazos_sw=cbind(588,270) #se establece el número de días que vivirá el swap
plazocupon_sw=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos_sw=cbind(1600,-1200) #se establece el número de contratos de cada swap
nominal_sw=cbind(1,1) #se establece el nominal de cada swap
por_sw=cbind(0,1) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable

#OPCIONES DE TASAS DE INTERÉS

#cargamos los factores de riesgo. Son dos curvas y una superficie de volatilidad.
btasadesc_oir="RiesgosFinancieros/2020-2/Tarea/tasa_TIIE_SW_OP.txt"
btasaspot_oir="RiesgosFinancieros/2020-2/Tarea/tasa_DIRS_SW_OP.txt"
bvolspot_oir="RiesgosFinancieros/2020-2/Tarea/tvoltiie_opc.txt"
plazos_oir=cbind( 1700, 700) #T-t, la temporalidad son días 
pr_oir=28 #plazo de referencia  
dct_oir=360 #d_base
cp_oir=cbind(1,0) #si es call (cap) o put (floor)
K_oir=cbind( 0.094, 0.089)
contratos_oir=cbind(1000, 500)
nominal_oir=1
cs_oir=1 #1 si es continua la tasa 0 si es simple


# Carga de datos

In [0]:
%%R

#CARGA DE DATOS DE ACCIONES
pos=cbind(t(pos_fx),t(pos_eq)) #cuanto compré de cada activo
start_date=Sys.Date()-nh #fecha inicial     [los valores de toman 3660 dias atrás de la fecha de la compu, o llegan hasta el 28 de feb???]
#   Creación del objeto para guardar los datos
dataEnv<-new.env()
dataEnvFX<-new.env()
#   obtener los datos
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)
getSymbols.yahoo(SymbolsFX,env=dataEnvFX,from=start_date)
#   limpiarlos, alinearnos y quedarnos con el precio de cierre 
bt.prep(dataEnv,align='remove.na',fill.gaps = T)
bt.prep(dataEnvFX,align='remove.na',fill.gaps=T)
#   Nos quedamos con los precios (ya los teníamos)
stock_prices = dataEnv$prices #acciones
stock_pricesFX=dataEnvFX$prices #divisas
#   cambiar todo a pesos mexicanos
stock_pricesFX=cbind(stock_pricesFX[,1,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,2,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,3,with=F])
stock_prices_EQFX=merge(stock_pricesFX,stock_prices,join = "inner")
x0=stock_prices_EQFX[nrow(stock_prices_EQFX),]
aux2_table=data.table(Date=as.Date(index(stock_prices_EQFX)),coredata(stock_prices_EQFX))

# x0_div=stock_pricesFX[nrow(stock_pricesFX),]
# aux_div=data.table(Date=as.Date(index(stock_pricesFX)),coredata(stock_pricesFX))

# x0_acc=stock_prices[nrow(stock_prices),]
# aux_acc=data.table(Date=as.Date(index(stock_prices)),coredata(stock_prices))


 aux_acc=aux2_table[,5:7]
 aux_div=aux2_table[,2:4]

stock_returns = Return.calculate(stock_prices_EQFX, method = c("continuous"))
#head(stock_returns[,1:3])
#print(head(stock_returns))

#CONFIRMACIÓN
DeltaX=as.matrix(log(as.matrix(stock_prices_EQFX[2:nrow(stock_prices_EQFX)])/as.data.table(as.matrix(stock_prices_EQFX[1:(nrow(stock_prices_EQFX)-1)]))))
#tail(DeltaX)
head(DeltaX)

#LEER DATOS DE CETES #CARGA DE DATOS DE BONO CUPÓN CERO
data<-read.table(base)
n<-nrow(data)
m_gov=ncol(data)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])

#LEER DATOS DE BONOS M
#carga de datos
#carga de tasas de descuento
data1<-read.table(btasadesc_bm)
n<-nrow(data1)
m_tybm=ncol(data1)
X_orig_tybm=as.data.table(mutate(data1[2:n,1:m_tybm],Date=as.Date(V1,format="%Y%m%d")))
X_orig_tybm=X_orig_tybm%>%select(-V1)
nodos_tybm=data.frame(data1[1,2:m_tybm])


##CARGA DE DATOS DE BONDE D
#carga de datos
#carga de tasas de descuento
data1<-read.table(base)
n<-nrow(data1)
m_bd=ncol(data1)
##X_orig_bd=as.data.table(mutate(data1[2:n,1:m_tybm],Date=as.Date(V1,format="%Y%m%d")))
#X_orig_bd%>%select(-V1)

X1_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
#nodos=data.frame(data1[1,2:m_bd])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,1:m3_bd],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)

X3a_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,1:2])
X2_orig_bd=mutate(X2_orig_bd, V1=as.Date(V1,format="%Y%m%d"), Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(min(X2_orig_bd$V1), max(X2_orig_bd$V1), "days") #sucesión de dias para tasa fondeo
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := V1][order(-Date)]
# rolling join unión por rolling, rellena las fechas que faltaban con el último valor conocido "roll=Inf"
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf] 
#buscar fecha de valuación en tfondeo
tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

X1_orig=setDT(X1_orig)[, Date:= V1][order(-Date)] #Para alinear con valor presente y tasa de fondeo.


#CARGA DE DATOS DE  FORWARDS DE TDC
#datas
#data<-read.table("tasa_tiie.txt")
data1<-read.table(bext)
data2<-read.table(bdom)

########minimos para parametrizar
n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1
###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])
####MATRICES DEL MISMO TAMAÑO MENOS DOLAR

x1_ftdc=as.data.table(mutate(data1[2:n,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)

###Para Dolar

if (yext==1)
{
  #Cargar los símbolos de yahoo finance para FX
  start_date=fval-3660 #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvFX<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsFX_ftdc,env=dataEnvFX,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvFX,align='remove.na',fill.gaps=T)
  
  #muestra de datos
  head(dataEnvFX$prices[,2])
  
  #Nos quedamos con los precios
  X3_ftdc=data.table(Date=as.Date(index(dataEnvFX$prices[,2])),coredata(dataEnvFX$prices[,2]))
} else  
{ 
data3<-read.table(btsp)
print(head(data3))
n3<-nrow(data3)
m3<-ncol(data3)
X3=data.table(as.matrix(as.double(as.matrix(data3[2:(n+1),m3]))))
X3_find=as.data.table(mutate(data3[2:(n+1),1:m3],Date=as.Date(V1,format="%Y%m%d")))
}


# CARGA DE DATOS DE  FORWARD DE IPC

data<-read.table(base)
n<-nrow(data)
m_gov=ncol(data)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])

  #Cargar los símbolos de yahoo finance para EQ
  start_date=fval-nh #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvEQ<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsEQ_find,env=dataEnvEQ,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvEQ,align='remove.na',fill.gaps=T)
  
  #muestra de datos
#  head(dataEnvEQ$prices)
  
  #Nos quedamos con los precios
  X3_find=data.table(Date=as.Date(index(dataEnvEQ$prices[,2])),coredata(dataEnvEQ$prices[,2]))

#CARGA DE DATOS PARA SWAP
  data1<-read.table(btasadesc_sw)
  n1<-nrow(data1)
  m1_orig_sw=ncol(data1)
  X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X1_orig_sw=X1_orig_sw%>%select(-V1)
  nodos1_sw=data.frame(data1[1,2:m1_orig_sw])
  
  data2<-read.table(btasacupvar_sw)
  n2<-nrow(data2)
  m2_orig_sw=ncol(data2)
  X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X2_orig_sw=X2_orig_sw%>%select(-V1)
  nodos2_sw=data.frame(data2[1,2:m2_orig_sw])
  
  #CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
  #carga de datos
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)
  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
  #data1[1:3,]
  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])
  

# Integración de Insumos

Aseguramiento de que todos los factores de riesgo estén alineados en sus fechas, nos quedamos con la intersección. Posteriormente, filtramos cada histórico factor de riesgo hacia esta intersección de todos.

In [0]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    

lin_gub=data.table(Date=as.Date(aux2_table[x_orig_gov,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity y guber
lin_gub=unique(lin_gub)
lin_gub_bmy=data.table(Date=as.Date(lin_gub[X_orig_tybm,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity y guber, con bonos M
lin_gub_bmy=unique(lin_gub_bmy)
lin_gub_bmybdst=data.table(Date=as.Date(lin_gub_bmy[X3_orig_bd,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber y st (bonde)
lin_gub_bmybdst=unique(lin_gub_bmybdst)
lin_gub_bmybdst_flib=data.table(Date=as.Date(lin_gub_bmybdst[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor
lin_gub_bmybdst_flib=unique(lin_gub_bmybdst_flib)
lin_gub_bmybdst_flibfwd=data.table(Date=as.Date(lin_gub_bmybdst_flib[x2_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd
lin_gub_bmybdst_flibfwd=unique(lin_gub_bmybdst_flibfwd)
lin_gub_bmybdst_flibfwdsp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwd[X3_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot
lin_gub_bmybdst_flibfwdsp=unique(lin_gub_bmybdst_flibfwdsp)
lin_gub_bmybdst_flibfwdspind=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdsp[X3_find,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index 
lin_gub_bmybdst_flibfwdspind=unique(lin_gub_bmybdst_flibfwdspind)
lin_gub_bmybdst_flibfwdspind_swcup=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind[X1_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón
lin_gub_bmybdst_flibfwdspind_swcup=unique(lin_gub_bmybdst_flibfwdspind_swcup)
lin_gub_bmybdst_flibfwdspind_swcupvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcup[X2_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón
lin_gub_bmybdst_flibfwdspind_swcupvp=unique(lin_gub_bmybdst_flibfwdspind_swcupvp)
lin_gub_bmybdst_flibfwdspind_swcupvp_oirs=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp[x1_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot
lin_gub_bmybdst_flibfwdspind_swcupvp_oirs=unique(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs)
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=unique(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp)
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=unique(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)

n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol) #Historia de todos
print(n)


[1] 214


In [0]:
%%R

#historia de acciones y divisas
stock_prices_EQFX=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[aux2_table,on=.(Date),nomatch=0][order(-Date)]
stock_prices_EQFX=stock_prices_EQFX%>%select(-Date)

#historia de curva gubernamental
x_orig_gov=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)

#historia de curva yield
X_orig_tybm=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X_orig_tybm,on=.(Date),nomatch=0][order(-Date)]
X_orig_tybm=X_orig_tybm%>%select(-Date)


#Historia de curvas de bonde
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X3_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X3_orig_bd=X3_orig_bd%>%select(-Date)


#historia de curvas de forward tdc
x1_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_ftdc,on=.(Date),nomatch=0][order(-Date)]
x1_ftdc=x1_ftdc%>%select(-Date)/100
x2_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_ftdc,on=.(Date),nomatch=0][order(-Date)]
x2_ftdc=x2_ftdc%>%select(-Date)/100
X3_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_ftdc,on=.(Date),nomatch=0][order(-Date)]
X3_ftdc=X3_ftdc%>%select(-Date)

#historia de curvas de forward ind
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X3_find=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_find=X3_find%>%select(-Date)

#historia de swaps
X1_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X1_orig_sw=X1_orig_sw%>%select(-Date)/100
X2_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X2_orig_sw=X2_orig_sw%>%select(-Date)/100

#historia de opciones
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)


# Pasos para medir el riesgo

##Delta-Normal
Para todos los instrumentos hay que definir y calcular los siguientes elementos:\
1.Historico de factores de riesgo\
2.Vector de precios actual\
3.Derivada con respecto al tiempo evaluada en el precio actual\
4.Primera derivada con respecto a factores de riesgo evaluada y multiplicada con su precio actual\

##Delta-Gamma
Consideramos los puntos anteriores y :\
5.Segunda derivada con respecto a factores de riesgo evaluada y multiplicada con sus precios actuales (al cuadrado)

##Delta-CF

#Función de valoración,derivadas y matriz hessiana

##ACCIONES Y DIVISAS

In [0]:
%%R
x0_acc_div=stock_prices_EQFX[1,]
#x0_acc= stock_prices[1,]
x0_acc=aux_acc[1,]
#stock_prices = dataEnv$prices #acciones
#stock_pricesFX=dataEnvFX$prices #divisas
x0_div=aux_div[1,]
#x0_div= stock_pricesFX[1,]

DeltaX_acc_div=as.matrix(log(as.matrix(stock_prices_EQFX[1:((stock_prices_EQFX)-1)])/as.data.table(as.matrix(stock_prices_EQFX[2:(nrow(stock_prices_EQFX))]))))
V0_acc_div=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
V0_acc=(t(pos_eq))*x0_acc
V0_div=(t(pos_fx))*x0_div

mu_acc_div=0
g_acc_div=cbind(t(pos_fx),t(pos_eq))
g_acc_divt=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
gamma_acc_divt=matrix(0,ncol(x0_acc_div),ncol(x0_acc_div))


m_fx=length(pos_fx)
m_acc=length(pos_eq)

DeltaX_div<- DeltaX_acc_div[,1:3]
DeltaX_acc<- DeltaX_acc_div[,4:6]

print(g_acc_divt)

R[write to console]: Error: NA/NaN argument

R[write to console]: In addition: 
R[write to console]: There were 11 warnings (use warnings() to see them)
R[write to console]: 




Error: NA/NaN argument


## CETES

In [0]:
%%R

#CETE CÁLCULO
#paso para interpolar las tasas
m=ncol(plazos_bcc)
x_bcc=matrix(0,n,m)
#Es necesario inicializar un ciclo el cual 
#     llenara la matriz "x_I" para cada contrato a valuar
for (i in 1:(n))
{
  #x[i,]=approx(nodos_gov,x_orig_gov[i,],plazos_bcc)$y
  x_bcc[i,]=if(itpl==0){approx(nodos_gov,x_orig_gov[i,],plazos_bcc,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[i,],plazos_bcc)}
}

x0_bcc=x_bcc[1,]

#Creamos una función de Valuación para el CETE
#     Dicha función debe recibir los datos iniciales
#     los cuales serán nuestros parámetros de la función
# 8.1.- Esta función DEBE RECIR:
#       a) plazos
#       b) Tipo de Operación
#       c) Valor Nominal
#       d) Tasa de rendimiento 

bonocupcero = function(i,t)
{
  1/(1+i*t/360)
}
V0_bcc=as.matrix(bonocupcero(x0_bcc,plazos_bcc))*contratos_bcc*nominal_bcc #Valor actual de cada bono

VT0_bcc=sum(V0_bcc) #Valor total del portafolio al tiempo 0
print("Valor total del portafolio al tiempo 0")
print(VT0_bcc)

#funcion primera derivada con tiempo
bcc_dt = function(i,t)
{
  -(i/360)/(1+i*t/360)^2
}

#funcion primera derivada con tasa interés
bcc_di = function(i,t)
{
  -(t/360)/(1+i*t/360)^2
}

#funcion segunda derivada con tasa de interés
bcc_di2 = function(i,t)
{
  2*(t/360)^2/(1+i*t/360)^3
}

#mu_bd
mu_bcc=bcc_dt(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc #primera derivada con respecto al tiempo evaluada en x0
g_bcc=bcc_di(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc #gradiente evaluado en x0
g_bcct=g_bcc*x0_bcc #gradiente evaluado en x0 por x0
gamma_bcc=bcc_di2(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc #evaluación de gamma
gamma_bcct2=diagv(gamma_bcc*(x0_bcc)^2) #evaluación de gamma multiplicado por x^{2}



[1] "Valor total del portafolio al tiempo 0"
[1] 14416.31


## Bonos M

In [0]:
%%R
#BONO M  CÁLCULO
#Posición inicial
#interpolación de tasas y volatilidades
m=ncol(plazos_bm)
X_bm=matrix(0,n,m)

#Funcion de valuación del BonoM
for (i in 1:(n))
{
  X_bm[i,]=if(itpl==0){approx(nodos_tybm,X_orig_tybm[i,],plazos_bm,rule=2)$y}else{talamb(nodos_tybm,X_orig_tybm[i,],plazos_bm)}
}

#Función de valoración por tasa yield
bonoMyield=function(x, plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #valoración bono tasa fija
{
  N=as.integer(plazos_bm/plazocupon_bm)+1   
  a=(1-(1+x*plazocupon_bm/360)^(-N))/(plazocupon_bm*x/360)
  p1=plazos_bm-plazocupon_bm*(N-1)   
  ((contratos_bm*nominal_bm*tfcupon_bm*plazocupon_bm/360)*a+(contratos_bm*nominal_bm)/((1+x*plazocupon_bm/360)^N))*(1+x*plazocupon_bm/360)^(1-p1/plazocupon_bm)   
}

x0_bm=t(as.matrix(X_bm[1,])) #tasas de descuento valor actual
V0_bm=bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm)

print("VALOR PRESENTE DEL BONO M")
print(V0_bm)


#derivada respecto al tiempo
 dbonoMyield_t=function(x, plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #valoración bono tasa fija
 {
   N=as.integer(plazos_bm/plazocupon_bm)+1   
   a=(1-(1+x*plazocupon_bm/360)^(-N))/(plazocupon_bm*x/360)
   p1=plazos_bm-plazocupon_bm*(N-1)   
   ((contratos_bm*nominal_bm*tfcupon_bm*plazocupon_bm/360)*a+(contratos_bm*nominal_bm)/((1+x*plazocupon_bm/360)^N))*(1+x*plazocupon_bm/360)^(1-p1/plazocupon_bm)*(x)^(3)  
 }

#gradiente
 dbonoMyield_x=function(x, plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #valoración bono tasa fija
 {
   N=as.integer(plazos_bm/plazocupon_bm)+1   
   a=(1-(1+x*plazocupon_bm/360)^(-N))/(plazocupon_bm*x/360)
   p1=plazos_bm-plazocupon_bm*(N-1)   
   ((contratos_bm*nominal_bm*tfcupon_bm*plazocupon_bm/360)*a+(contratos_bm*nominal_bm)/((1+x*plazocupon_bm/360)^N))*(1+x*plazocupon_bm/360)^(1-p1/plazocupon_bm)*(plazos_bm/360)*x^(2)   
 }

#gamma
 dbonoMyield_x2=function(x, plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #valoración bono tasa fija
 {
   N=as.integer(plazos_bm/plazocupon_bm)+1   
   a=(1-(1+x*plazocupon_bm/360)^(-N))/(plazocupon_bm*x/360)
   p1=plazos_bm-plazocupon_bm*(N-1)   
   ((contratos_bm*nominal_bm*tfcupon_bm*plazocupon_bm/360)*a+(contratos_bm*nominal_bm)/((1+x*plazocupon_bm/360)^N))*(1+x*plazocupon_bm/360)^(1-p1/plazocupon_bm)*(plazos_bm/360)^2*x^(4)    
 }

mu_bm=dbonoMyield_t(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #primera derivada con respecto al tiempo evaluada en x0
g_bm=dbonoMyield_x(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #gradiente evaluado en x0
g_bmt=g_bm*x0_bm #gradiente evaluado en x0 multiplicado por x0
gamma_bm=dbonoMyield_x2(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #evaluación de gamma
gamma_bmt2=diagv(gamma_bm*(x0_bm)^2) #evaluación de gamma multiplicado por x^2


[1] "VALOR PRESENTE DEL BONO M"
          [,1]
[1,] -975415.9


## Bondes D

In [0]:
%%R
#BONDE D CÁLCULO

X2_pr=lin_gub_bmy[X2_orig_bd, on = .(Date),nomatch=0][order(-Date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
m=ncol(plazos_bdm)


N_bd=as.integer(plazos_bdm/plazocupon_bdm)+1 #número de cupones a pagar
VTplazos_bdm=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdm de todos los contratos_bdm
contratos_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los contratos_bdm de todos los flujos de todos los contratos_bdm
nominal_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los nominal_bdmes de todos los flujos de todos los contratos_bdm
plazocupon_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdmcupon de todos los flujos de todos los contratos_bdm
tasafijaT_bd=matrix(0,1,sum(N_bd)) #vector de tasas fijas de todos los flujos de todos los contratos_bdm
ulNomT_bd=matrix(0,1,sum(N_bd)) #vector de contratos_bdm a final de flujo


plazini_bd=plazos_bdm-plazocupon_bdm*(N_bd-1) #vector de plazos_bdm iniciales
ddv=plazocupon_bdm-plazini_bd #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos

#calcula cupones de bonos
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_bdm[,1:sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,1:sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])
    plazocupon_bdmT[,1:sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
    tasafijaT_bd[,1]=tfcupon[j]
    tasafijaT_bd[,2:sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos_bdm[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])					
    plazocupon_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    tasafijaT_bd[,(sum(N_bd[1:j-1])+1)]=tfcupon[j]
    tasafijaT_bd[,(sum(N_bd[1:j-1])+2):sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
  }
}

Xvp_bd=matrix(0,n,ncol(VTplazos_bdm))
Xst_bd=matrix(0,n,ncol(VTplazos_bdm))

for (i in (1:n))
{
  Xvp_bd[i,]=if(itpl==0){approx(nodos_gov,x_orig_gov[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[i,],VTplazos_bdm)}
  Xst_bd[i,]=if(itpl==0){approx(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm)}
  
}


bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  tasafijaT=matrix(0,1,sum(N))

  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  #valor del portafolio a tiempo cero
  V0=matrix(0,1,count(N))
  V0f=((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 


V0_bdm=bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
print("VALOR DEL PORTAFOLIO DE BONDES EN TIEMPO CERO")
print(V0_bdm)

#3.Derivada con respecto al tiempo evaluada en el precio actual
#4.Primera derivada con respecto a factores de riesgo evaluada y multiplicada con su precio actual
#5.Segunda derivada con respecto a factores de riesgo evaluada y multiplicada con sus precios actuales (al cuadrado)\

#Derivada con respecto al tiempo evaluada en el precio actual
d_bondeD_t=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=-((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((Xvp+Xst)/360)
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 


#la derivada con respecto al valor presente y a la sobretasa es igual
d_bondeD_x=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  V0f=-((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((VTplazos_bdm)/360)
  V0f
} 


d_bondeD_tc=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  V0f=-((((contratos_bdmT*(plazocupon_bdmT/360)))/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  V0f
}


#SEGUNDA DERIVADA
mgamma_bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)	#matriz de segundas derivadas parciales de la tasa valor presente y pago de flujos


{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }

  c11=matrix(0,1,length(contratos_bdmT)) #segunda derivada del cupón
  c12=-((((contratos_bdmT*(plazocupon_bdmT/360)))/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((VTplazos_bdm)/360) #primera del cupón y primera del vp
  c13=c12 #primera del cupón y primera del st
  c21=-((((contratos_bdmT*(plazocupon_bdmT/360)))/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((VTplazos_bdm)/360) #primera del vp y primera del cupon
  c22=2*((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)^3))*nominal_bdm*((VTplazos_bdm)/360)^2 #segunda del vp
  c23=c22 #primera del vp y primera del st
  c31=c12 #primera del st y primera del cupon
  c32=c22 #primera del vp y primera del st
  c33=c22 #segunda del st
  rbind(cbind(diagv(c11),diagv(c12), diagv(c13)),cbind(diagv(c21),diagv(c22), diagv(c23)),cbind(diagv(c31),diagv(c32), diagv(c33)))
}


#mgamma_bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)

X_bd_tc=matrix(1,n,ncol(contratos_bdmT))*X2_pr$V2/100 #Derivada con respecto al tiempo evaluada en el precio actual
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp_bd),as.matrix(Xst_bd)) #Primera derivada con respecto a factores de riesgo evaluada y multiplicada con su precio actual
Delta_X_bd_ext=X_bd_ext[1:(n-1),]/X_bd_ext[2:(n),]-1 #5.Segunda derivada con respecto a factores de riesgo evaluada y multiplicada con sus precios actuales (al cuadrado)\

mu_bd=d_bondeD_t(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv) #mu para normal
g_bd=cbind(d_bondeD_tc(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv),d_bondeD_x(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv),d_bondeD_x(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv))
x0_bd_ext=cbind(matrix(tf_act,1,ncol(contratos_bdmT)),t(as.matrix(Xvp_bd[1,])),t(as.matrix(Xst_bd[1,])))
g_bdt=g_bd*x0_bd_ext 
gamma_bd=mgamma_bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
gamma_bdt2=gamma_bd*(t(x0_bd_ext)%*%x0_bd_ext)
VarCovX_bd_ext=cov(Delta_X_bd_ext)*(n-1)/(n-2) #cov de segunda derivada
tgammavar_bdt=(gamma_bdt2%*%VarCovX_bd_ext) #matriz de var y covar
tgammavar2_bdt=(gamma_bdt2%*%VarCovX_bd_ext)%*%(gamma_bdt2%*%VarCovX_bd_ext) 


[1] "VALOR DEL PORTAFOLIO DE BONDES EN TIEMPO CERO"
         [,1]
[1,] 95226.61


##FORWARDS TDC

In [0]:
%%R
#FORWARDS Y/O FUTUROS DE TIPO DE CAMBIO CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################
#FORWARDS Y/O FUTUROS DE TIPO DE CAMBIO CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m=ncol(plazos_fwd)
X1_fwtdc=matrix(0,n,m)
X2_fwtdc=matrix(0,n,m)

for (j in 1:n)
{
  X1_fwtdc[j,]=if(itpl==0){approx(nodos1_ftdc,x1_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwtdc[j,]=if(itpl==0){approx(nodos2_ftdc,x2_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos2_ftdc,x2_ftdc[j,],plazos_fwd)}
  if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}


futuroTC = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

X3_ftdc=as.matrix(X3_ftdc)

V0_fwtdc=futuroTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd

print("VALOR ACTUAL")
print(V0_fwtdc)

dfuturoTC_tl = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=-s*((1+tn*t/360)/(1+tl*t/360)^2)*(t/360) #Se obtiene el tipo de cambio forward
  t(as.numeric((f)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_tn = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  f1=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f)/(1+t*tn/360)-(f1-k)/(1+t*tn/360)^2*(t/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_s = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  1/(1+t*tl/360) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_t = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn/360)/(1+tl*t/360))-s*((1+tn*t/360)/(1+tl*t/360)^2)*(tl/360) #Se obtiene el tipo de cambio forward
  #t(as.numeric((f)/(1+t*tn/360)-(f-k)/(1+t*tn/360)^2*(tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
  -(f-k)/(1+t*tn/360)^2*(tn/360)
}

mgamma_futTC= function(t,tl,tn,s,k,contratos_fwd,nominal_fwd)
{
c11= matrix(0,1,length(k))#spot spot
c12= -(((1+tn*t/360)/(1+tl*t/360)^2)*(t/360))/(1+t*tn/360)*contratos_fwd*nominal_fwd#spot tl
c13=  matrix(0,1,length(k))  #spot tn
c21=  -(((1+tn*t/360)/(1+tl*t/360)^2)*(t/360))/(1+t*tn/360)*contratos_fwd*nominal_fwd #tl spot
c22= (2*s*((1+tn*t/360)/(1+tl*t/360)^3)*(t/360)^3)/(1+t*tn/360)*contratos_fwd*nominal_fwd #tl tl
c23= matrix(0,1,length(k)) #tl tn
c31=(((t/360)/(1+tl*t/360))/(1+t*tn/360)-(((1+tn*t/360)/(1+tl*t/360)))/(1+t*tn/360)^2*(t/360))*contratos_fwd*nominal_fwd #tn spot
c32=   (-s*((t/360)^2/(1+tl*t/360)^2)/(1+t*tn/360)-(s*((t/360)/(1+tl*t/360)))/(1+t*tn/360)^2*(t/360))*contratos_fwd*nominal_fwd #tn tl
c33=  (s*((t/360)/(1+tl*t/360))/(1+t*tn/360)^2*(t/360)+2*(s*((1+tn*t/360)/(1+tl*t/360))-k)/(1+t*tn/360)^3*(t/360)^2-(s*((t/360)^2/(1+tl*t/360)))/(1+t*tn/360)^2)*contratos_fwd*nominal_fwd #tn tn
m=length(t)
rbind(cbind(diagv(c11),diagv(c12),diagv(c13)),cbind(diagv(c21),diagv(c22),diagv(c23)),cbind(diagv(c31),diagv(c32),diagv(c33)))
}


X_ftdc=cbind(matrix(X3_ftdc,n,ncol(X1_fwtdc)),X1_fwtdc,X2_fwtdc) #Hist facto riesgo
xp_ftdc=t(as.matrix(X_ftdc[1,])) #Valores actuales X0
mu_ftdc=dfuturoTC_t(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd #primera derivada con respecto al tiempo evaluada en x0
g_ftdc=cbind(dfuturoTC_s(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd,dfuturoTC_tl(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd,dfuturoTC_tn(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd) #gradiente evaluado en x0
g_ftdctT=xp_ftdc*g_ftdc #gradienete evaluado en x0 por x0
gamma_ftdc=mgamma_futTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd,contratos_fwd,nominal_fwd) #evaluación de gamma
gamma_ftdctT2=gamma_ftdc*(t(xp_ftdc)%*%xp_ftdc) #evaluación de gamma multiplicado por x^{2}


[1] "VALOR ACTUAL"
          [,1]
[1,] -53.23399


## FORWARDS DE INDICES

In [0]:
%%R
#FORWARDS Y/O FUTUROS DE ÍNDICES CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m=ncol(plazos_fwd_ind)
X1_fwind=matrix(0,n,m) #DIVIDENDOS
X2_fwind=matrix(0,n,m)

for (j in 1:n)
{
  #X1_fwind[j,]=if(itpl==0){approx(nodos1_,x1_ftdc[j,],plazos_fwd)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwind[j,]=if(itpl==0){approx(nodos_gov,x_orig_gov[j,],plazos_fwd_ind,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[j,],plazos_fwd_ind)}
  #if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}

X3_find=as.matrix(X3_find)

V0_fwind=futuroTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind

print("VALOR ACTUAL")
print(V0_fwind)

X_find=cbind(matrix(X3_find,n,ncol(X1_fwind)),X1_fwind,X2_fwind)
xp_find=t(as.matrix(X_find[1,]))
mu_find=dfuturoTC_t(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
g_find=cbind(dfuturoTC_s(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind,dfuturoTC_tl(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind,dfuturoTC_tn(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind)
g_findtT=xp_find*g_find
gamma_find=mgamma_futTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind,contratos_fwd_ind,nominal_fwd_ind)
gamma_findtT2=gamma_find*(t(xp_find)%*%xp_find)


[1] "VALOR ACTUAL"
        [,1]
[1,] 4528398


##SWAPS


In [0]:
%%R
##SWAP TASA FIJA VS TASA VARIABLE CÁLCULO

##Interpolamos

nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=as.matrix(X1_orig_sw)
curvatc=X2_orig_sw
n1=nrow(curvavp)
n2=nrow(curvatc)


m=max(ncol(plazos_sw),1) #número de contratos_sw swap a valorar
N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato
for (j in (1:m))
{
  N[j]=as.integer(plazos_sw[j]/plazocupon_sw[j])+1 #número de cupones a pagar
}
VTplazos_sw=matrix(0,1,sum(N)) #vector de todos los plazos_sw de todos los contratos_sw
contratos_swT=matrix(0,1,sum(N)) #vector de todos los contratos_sw de todos los flujos de todos los contratos_sw
nominal_swT=matrix(0,1,sum(N)) #vector de todos los nominal_swes de todos los flujos de todos los contratos_sw
por_swT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT=matrix(0,1,sum(N)) #vector de todos los plazos_swcupon de todos los flujos de todos los contratos_sw
tasafija_swT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos_sw
VTplazos_swc=matrix(0,1,sum(N)) #vector de todos los plazos_sw cortos de todos los contratos_sw


plazini=plazos_sw-plazocupon_sw*(N-1) #vector de plazos_sw iniciales

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_sw[,1:sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,1:sum(N[1:j])]=c(0,VTplazos_sw[,1:(sum(N[1:j])-1)])
    contratos_swT[,1:sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])
    nominal_swT[,1:sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,1:sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,1:sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,1:sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
  else
  {
    VTplazos_sw[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos_sw[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratos_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])					
    nominal_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
}


Xvp=matrix(0,n,ncol(VTplazos_sw))
Xtc=matrix(0,n,ncol(VTplazos_sw))
Xtcc=matrix(0,n,ncol(VTplazos_sw))
XtfwdT=matrix(0,n,ncol(VTplazos_sw))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodosvp,curvavp[i,],VTplazos_sw,rule=2)$y}else{talamb(nodosvp,curvavp[i,],VTplazos_sw)}
  Xtc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_sw,rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_sw)}
  Xtcc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_swc, rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_swc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos_sw/360)/(1+Xtcc[i,]*VTplazos_swc/360)-1)*360/plazocupon_swT 
  for (j in (1:ncol(VTplazos_sw)))
  {
    if (VTplazos_sw[j]<= plazocupon_swT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}


swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT/360)))/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}

V0_sw=swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,], N)


#derivada del swap con respecto aL tiempo
dswap_dtt=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  -1*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*Xvp/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
}

#derivada del swap con respecto a la tasa cupón variable
dswap_dtc=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  (((contratos_swT*plazocupon_swT)/360)/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
}

#derivada del swap con respecto al valor presente
dswap_dvp=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  -1*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*VTplazos_sw/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
}

mgamma_swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)	#matriz de segundas derivadas parciales de la tasa valor presente y pago de flujos
{
  c11=matrix(0,1,ncol(contratos_swT))
  c21=-1*(((contratos_swT*(plazocupon_swT*VTplazos_sw/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
  c12=-1*(((contratos_swT*plazocupon_swT*VTplazos_sw)/360^2)/(1+Xvp*VTplazos_sw/360)^2)*nominal_swT*(-1)^por_swT
  c22=2*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*VTplazos_sw^2/360^3)))/(1+Xvp*VTplazos_sw/360)^(3))*nominal_swT*(-1)^por_swT
  rbind(cbind(diagv(c11),diagv(c12)),cbind(diagv(c21),diagv(c22)))
}

X_sw=cbind( XtfwdT,Xvp)

mu_sw=sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]))
g_sw=cbind(dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]),dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]))


#Multiplicación de la derivada por el punto x0
gt_sw=c( XtfwdT[1,],Xvp[1,])*g_sw

gamma_sw=mgamma_swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,])
tgamma_sw=c( XtfwdT[1,],Xvp[1,])%*%t(c( XtfwdT[1,],Xvp[1,]))*gamma_sw 
print("Valor Actual")
print(V0_sw)

[1] "Valor Actual"
         [,1]     [,2]
[1,] 63.75999 28.39141


##OPCIONES DE TASAS DE INTERÉS

In [0]:
%%R
##opciones de tasa de interés, con inicio el día de la valuación CÁLCULO
#Posición inicial

#interpolación de tasas y volatilidades
m_opc=ncol(plazos_oir)  #m significa el número de contratos de opciones van a calcular
#factores de riesgo:
x1=matrix(0,n,m_opc)    
x2tc=matrix(0,n,m_opc) #x2 es el factor de riesgo tasa forwar, lo dividimos en tasa larga y tasa corta.
x2tl=matrix(0,n,m_opc)
x2=matrix(0,n,m_opc)
x3=matrix(0,n,m_opc)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3[i,]=if(itpl==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}


x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades


#procedemos a valorar la opción tdi según los datos de la presentación
opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
  #cs_oir=1 #1 si es continua la tasa 0 si es simple
}

#valoración de opciones tasa de interés
V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio
 
#DERIVAMOS B&S

#pr_oirIMERA DERIVADA

dopctintt = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y TIEMPO
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(( -(S*dnorm(d1)*vol)/(2*sqrt(t/365))+(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(t/365)*(-1)^cp_oir)*(exp(-vp*t/365)))}else{(( -(S*dnorm(d1)*vol)/(2*sqrt(t/360))+(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(t/360)*(-1)^cp_oir)*(1/(1+vp*t/360)^2))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}
#dopctintt(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

dopctintS = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y spot
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){((pnorm(d1*(-1)^cp_oir))*(exp(-vp*t/365)))}else{(pnorm(d1*(-1)^cp_oir)*(1/(1+vp*t/360)))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)- (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2
}
#dopctintS(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

dopctintvol = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y spot (volatilidad)
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
  (if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}
#dopctintvol(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

dopctintrho = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y rho (tasa de interés)
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
  (if(cs_oir==1){-(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(t/365)}else{-(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^2)*(-1)^cp_oir*(t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}
#dopctintrho(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

#matríz hessiana (cuadrada, de 3x3)
mgammaopc=function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir)	#matriz de segundas derivadas parciales de la call: rho, spot y vol de spot
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
  c11=(if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(t/365)^2}else{2*(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^3)*(-1)^cp_oir*(t/360)^2})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #rho con respecto a rho
  c21=(if(cs_oir==1){((pnorm(d1*(-1)^cp_oir))*(exp(-vp*t/365)))*(-t/365)}else{(pnorm(d1*(-1)^cp_oir)*(1/(1+vp*t/360)^2))*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)- (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(-t/365)}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^2)*(-1)^cp_oir*(-t/360)})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2*contratos_oir*nominal_oir #spot con respecto a rho
  c31=(if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))*(-t/365)}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360)^2)*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #vol con respecto a rho
  c12=c21 #rho con respecto a spot
  c22=(if(cs_oir==1){(dnorm(d1)/(S*vol*sqrt(t/365)))*(exp(-vp*t/365))}else{(dnorm(d1)/(S*vol*sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir) -2*((if(cs_oir==1){(((pnorm(d1*(-1)^cp_oir))+( S*dnorm(d1*(-1)^cp_oir)-K_oir*dnorm(d2*(-1)^cp_oir)))*(exp(-vp*t/365)))*(-1)^cp_oir*(1/(S*vol*sqrt(t/365)))}else{(((pnorm(d1*(-1)^cp_oir))+( S*dnorm(d1*(-1)^cp_oir)-K_oir*dnorm(d2*(-1)^cp_oir)))*(1/(1+vp*t/360)))*(1/(S*vol*sqrt(t/360)))*(-1)^cp_oir})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2) +2*(if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)^3/(1+S*pr_oir/dct_oir)^3*contratos_oir*nominal_oir #spot spot 
  c32=(if(cs_oir==1){dnorm(d1)*d2/vol*(exp(-vp*t/365))}else{dnorm(d1)*d2/vol*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir) +   (if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2*contratos_oir*nominal_oir # vol spot
  c13=(if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))*(-t/365)}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360)^2)*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #rho vol
  c23=c32 #spot vol
  c33=(if(cs_oir==1){(dnorm(d1)*(S*sqrt(t/365)))*((d1*d2)/vol)*exp(-vp*t/365)}else{(dnorm(d1)*(S*sqrt(t/360)))*((d1*d2)/vol)*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #vol vol
  # outm=matrix(0,3,length(plazos_oir)*3)
  m_opc=length(t)
 rbind(cbind(diagv(c11),diagv(c12),diagv(c13)),cbind(diagv(c21),diagv(c22),diagv(c23)),cbind(diagv(c31),diagv(c32),diagv(c33)))
}

#mgammaopc(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir)

X_oir=cbind(x1,x2,x3)                   #ponemos en un vector los factores de riesgo, en total hay X_oir
xp_oir=cbind(t(x01),t(x02),t(x03))      #valor actual de los X0
mu_oir=dopctintt(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir         #derivada con respecto al tiempo
delta_oir_T=cbind(dopctintrho(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir, dopctintS(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir,dopctintvol(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir)      #Primera derivada (gradiente) evaluado en el punto X0
deltaT_oir_T=xp_oir*delta_oir_T       #el gradiente multiplicado por el factor de riesgo, por X0
gammaT_oir=mgammaopc(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir) #la matríz hessiana evaluada en X0
gammaT_oirT2=gammaT_oir*(t(xp_oir)%*%xp_oir)    #la hessiana multiplicada por X0^2

print(xp_oir)

           [,1]       [,2]       [,3]     [,4]      [,5]      [,6]
[1,] 0.09438377 0.08723297 0.05901016 0.065173 0.2979286 0.2863077


# Integración de factores y cálculo de riesgo en conjunto

In [0]:
%%R
#ACCIONES Y DIVISAS
n_if_acc_div=matrix(0,1,1)
n_if_acc_div[1]=ncol(stock_prices_EQFX)
n_acc<-length(stock_prices_EQFX)

m_if=matrix(0,1,1)
m_if[1]=ncol(DeltaX_acc)
m<-length(DeltaX_acc)

DeltaX_acc_div=as.matrix(log(stock_prices_EQFX[1:(n-1)]/stock_prices_EQFX[2:(n)]))
DeltaX_acc_div[is.nan(DeltaX_acc_div)] <- 0 #quitamos NaN
DeltaX_acc_div[is.na(DeltaX_acc_div)] <- 0 #quitamos Na
DeltaX_acc_div[is.infinite(DeltaX_acc_div)] <- 0 #quitamos Na

#Acciones
DeltaX_acc[is.nan(DeltaX_acc)]<-0
DeltaX_acc[is.na(DeltaX_acc)] <- 0 #quitamos Na
DeltaX_acc[is.infinite(DeltaX_acc)] <- 0 #quitamos Na

#Divisas
DeltaX_div[is.nan(DeltaX_div)]<-0
DeltaX_div[is.na(DeltaX_div)] <- 0 #quitamos Na
DeltaX_div[is.infinite(DeltaX_div)] <- 0 #quitamos Na

#Cálculo de matriz de varianza covarianza de todos los factores de riesgo del Acciones y divisas
VarDeltaX_acc_div=as.matrix(cov(DeltaX_acc_div)*(n_acc-1)/(n_acc-2))
VarDeltaX_acc=as.matrix(cov(DeltaX_acc)*(n_acc-1)/(n_acc-2))
VarDeltaX_div=as.matrix(cov(DeltaX_div)*(n_acc-1)/(n_acc-2))


#CETES, BONDES D Y BONOS M
n_if=matrix(0,3,1)
n_if[1]=ncol(x_bcc) #cetes
n_if[2]=ncol(X_bm) #bonos m
n_if[3]=ncol(X_bd_ext) #bonde

#valor del portafolios

V0_port=cbind(V0_bcc,V0_bm, V0_bdm)
V0T_port=sum(V0_port)

#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(x_bcc,X_bm,X_bd_ext) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIO
DeltaX_port=as.matrix(log(X_port[1:(n-1),]/X_port[2:(n),]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na

#FORWARDS TDC E IND
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if_forwards=matrix(0,2,1)
n_if_forwards[1]=ncol(X_ftdc) #fwd tdc
n_if_forwards[2]=ncol(X_find) #fwd ind

#valor del portafolios

V0_port_forwards=cbind(V0_fwtdc, V0_fwind)
V0T_port_forwards=sum(V0_port_forwards)



#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port_forwards=cbind(X_ftdc,X_find) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port_forwards=as.matrix(log(X_port_forwards[1:(n-1),]/X_port_forwards[2:(n),]))
DeltaX_port_forwards[is.nan(DeltaX_port_forwards)] <- 0 #quitamos NaN
DeltaX_port_forwards[is.na(DeltaX_port_forwards)] <- 0 #quitamos Na
DeltaX_port_forwards[is.infinite(DeltaX_port_forwards)] <- 0 #quitamos Na

#DIMENSION DE TODOS LOS INSTRUMENTOS
n_if_opc=matrix(0,1,1)      #son opciones, por eso en nrow es 1, las columnas serán el número total de factores de riesgo que hay (línea siguiente)
n_if_opc[1]=ncol(X_oir)     #ncol(X_oir) es 6, que son los faxctores de riesgo

#valor del portafolios
V0_port_opc=cbind(V0_oir)   #valor actual del portafolio
V0T_port_opc=sum(V0_port_opc)

#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port_opc=cbind(X_oir) #Factores de riesgo del portafolios (son 3 por cada contrato)

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port_opc=as.matrix(log(X_port_opc[1:(n-1),]/X_port_opc[2:(n),]))
DeltaX_port_opc[is.nan(DeltaX_port_opc)] <- 0 #quitamos NaN
DeltaX_port_opc[is.na(DeltaX_port_opc)] <- 0 #quitamos Na
DeltaX_port_opc[is.infinite(DeltaX_port_opc)] <- 0 #quitamos Na

#SWAPS NORMAL     
DeltaXvp=Xvp[1:(n-1),]/Xvp[2:n,]-1
DeltaXtfwdt=XtfwdT[1:(n-1),]/XtfwdT[2:n,]-1

DeltaX=cbind(DeltaXtfwdt,DeltaXvp)

VarCovX_sw=cov(DeltaX)*((n-1)/(n-2))  #Matriz de Varianza-Covarianza
ncol(VarCovX_sw)

#SWAPS CF Y DELTA GAMMA NORMAL

#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if_sw=matrix(0,1,1)
n_if_sw[1]=ncol(X_sw) #swaps

#valor del portafolios

V0_port_sw=cbind(V0_sw)
V0T_port_sw=sum(V0_port_sw)


#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port_sw=cbind(X_sw) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port_sw=as.matrix(log(X_port_sw[1:(n-1),]/X_port_sw[2:(n),]))
DeltaX_port_sw[is.nan(DeltaX_port_sw)] <- 0 #quitamos NaN
DeltaX_port_sw[is.na(DeltaX_port_sw)] <- 0 #quitamos Na
DeltaX_port_sw[is.infinite(DeltaX_port_sw)] <- 0 #quitamos Na

print(n)
print(head(X_port_opc))
print(head(X_port_opc))




[1] 214
           [,1]       [,2]       [,3]       [,4]      [,5]      [,6]
[1,] 0.09438377 0.08723297 0.05901016 0.06517300 0.2979286 0.2863077
[2,] 0.09477752 0.08741573 0.06019018 0.06459820 0.2979286 0.2863846
[3,] 0.09520313 0.08750312 0.05982924 0.06363872 0.2979286 0.2863846
[4,] 0.09582160 0.08813738 0.05936498 0.06486578 0.2979286 0.2836154
[5,] 0.09574553 0.08804257 0.06109638 0.06449413 0.2979286 0.2836154
[6,] 0.09490292 0.08740767 0.06039584 0.06471889 0.2979286 0.2835385
           [,1]       [,2]       [,3]       [,4]      [,5]      [,6]
[1,] 0.09438377 0.08723297 0.05901016 0.06517300 0.2979286 0.2863077
[2,] 0.09477752 0.08741573 0.06019018 0.06459820 0.2979286 0.2863846
[3,] 0.09520313 0.08750312 0.05982924 0.06363872 0.2979286 0.2863846
[4,] 0.09582160 0.08813738 0.05936498 0.06486578 0.2979286 0.2836154
[5,] 0.09574553 0.08804257 0.06109638 0.06449413 0.2979286 0.2836154
[6,] 0.09490292 0.08740767 0.06039584 0.06471889 0.2979286 0.2835385


#Medición de Riesgo

## RIESGO ACCIONES Y DIVISAS

In [0]:
%%R
mu_acc_div=as.matrix(mu_acc_div) #primera derivada con respecto al tiempo
#INTEGRACIÓN DE PRIMERAS DERIVADAS (GRADIENTE)
gt_acc_div=as.matrix(g_acc_divt) #derivada con respecto las X's
VaR_por_div=matrix(0,1,3)
CVaR_por_div=matrix(0,1,3)

VaR_por_acc=matrix(0,1,3)
CVaR_por_acc=matrix(0,1,3)

for(i in 1:3){
   w_fr=diagv(c(1,1,1))
   
  gt_div_n=as.matrix(gt_acc_div[,i])*w_fr[i,]
  gt_acc_n=as.matrix(gt_acc_div[,i+3])*w_fr[i,]
  mu_acc_div_dn=0

  VaR_por_div[i]=mu_acc_div_dn+sqrt(t(gt_div_n)%*%VarDeltaX_div%*%(gt_div_n))*qnorm(1-alpha)
  CVaR_por_div[i]=-mu_acc_div_dn-sqrt(t(gt_div_n)%*%VarDeltaX_div%*%(gt_div_n))*(dnorm(qnorm(alpha))/(1-alpha))

  VaR_por_acc[i]=mu_acc_div_dn+sqrt(t(gt_acc_n)%*%VarDeltaX_acc%*%(gt_acc_n))*qnorm(1-alpha)
  CVaR_por_acc[i]=-mu_acc_div_dn-sqrt(t(gt_acc_n)%*%VarDeltaX_acc%*%(gt_acc_n))*(dnorm(qnorm(alpha))/(1-alpha))

}
print("VaR y CVaR por divisa")
print(VaR_por_div/sum(V0_div))
print(CVaR_por_div/sum(V0_div))

print("VaR y CVaR por acc")
print(VaR_por_acc/sum(V0_acc))
print(CVaR_por_acc/sum(V0_acc))

#INTEGRACIÓN DE MATRIZ HESSIANA
gammat_acc_div=matrix(0,sum(n_if_acc_div),sum(n_if_acc_div))
gamma_acc_divt=gammat_acc_div[1:n_if_acc_div[1],1:n_if_acc_div[1]]

gammat_acc=gammat_acc_div[1:3,1:3]
gamma_acc=gamma_acc_divt[1:3,1:3]

gammat_div=gammat_acc
gamma_div=gamma_acc

tgammavar_acc_div=(gamma_acc_divt%*%VarDeltaX_acc_div)
tgammavar2_acc_div=(tgammavar_acc_div)%*%(tgammavar_acc_div) 

#Acciones
tgammavar_acc=(gamma_acc%*%VarDeltaX_acc)
tgammavar2_acc=(tgammavar_acc)%*%(tgammavar_acc)

#Divisas
tgammavar_div=(gamma_div%*%VarDeltaX_div)
tgammavar2_div=(tgammavar_div)%*%(tgammavar_div)

#Acciones y divisas
VaRT_acc_div_DGM=(sum(diag(tgammavar_acc_div))/2+sum(mu_acc_div))-(sqrt(sum(diag(tgammavar2_acc_div))/2+gt_acc_div%*%VarDeltaX_acc_div%*%t(gt_acc_div)))*qnorm(alpha) #VaR Normal
CVaRT_acc_div_DGM=(sum(diag(tgammavar_acc_div))/2+sum(mu_acc_div))-(sqrt(sum(diag(tgammavar2_acc_div))/2+gt_acc_div%*%VarDeltaX_acc_div%*%t(gt_acc_div)))*dnorm(qnorm(alpha))/(1-alpha)

print("Total acciones y divisas")
print(VaRT_acc_div_DGM/sum(V0_acc_div))
print(CVaRT_acc_div_DGM/sum(V0_acc_div))

gt_acc=as.matrix(gt_acc_div[,4:6])
gt_div=as.matrix(gt_acc_div[,1:3])

#Acciones
VaRT_acc_DGM=(sum(diag(tgammavar_acc))/2+sum(mu_acc_div))-(sqrt(sum(diag(tgammavar2_acc))/2+t(gt_acc)%*%VarDeltaX_acc%*%(gt_acc)))*qnorm(alpha) #VaR Normal
CVaRT_acc_DGM=(sum(diag(tgammavar_acc))/2+sum(mu_acc_div))-(sqrt(sum(diag(tgammavar2_acc))/2+t(gt_acc)%*%VarDeltaX_acc%*%(gt_acc)))*dnorm(qnorm(alpha))/(1-alpha)

print("Total acciones")
print(VaRT_acc_DGM/sum(V0_acc))
print(CVaRT_acc_DGM/sum(V0_acc))

#Divisas
VaRT_div_DGM=(sum(diag(tgammavar_div))/2+sum(mu_acc_div))-(sqrt(sum(diag(tgammavar2_div))/2+t(gt_div)%*%VarDeltaX_div%*%(gt_div)))*qnorm(alpha) #VaR Normal
CVaRT_div_DGM=(sum(diag(tgammavar_div))/2+sum(mu_acc_div))-(sqrt(sum(diag(tgammavar2_div))/2+t(gt_div)%*%VarDeltaX_div%*%(gt_div)))*dnorm(qnorm(alpha))/(1-alpha)

print("Total divisas")
print(VaRT_div_DGM/sum(V0_div))
print(CVaRT_div_DGM/sum(V0_div))

###VaR Cornish Fisher
tgammavar3_acc_div=tgammavar2_acc_div%*%(tgammavar_acc_div)
asimP_acc_div=sum(diag(tgammavar3_acc_div))+3*gt_acc_div%*%VarDeltaX_acc_div%*%gammat_acc_div%*%VarDeltaX_acc_div%*%t(gt_acc_div) #gamma tercer momento
asimP_acc_div=asimP_acc_div/(sqrt(sum(diag(tgammavar2_acc_div))/2+gt_acc_div%*%VarDeltaX_acc_div%*%t(gt_acc_div)))^3  #coef de asimetría
VaRstd_acc_div=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_acc_div  #VaR estandarizado
VaR_acc_div_DGM_CF=(sum(diag(tgammavar_acc_div))/2+sum(mu_acc_div))+(sqrt(sum(diag(tgammavar2_acc_div))/2+gt_acc_div%*%VarDeltaX_acc_div%*%t(gt_acc_div)))*VaRstd_acc_div#VaR cf
CVaR_acc_div_DGM_CF=(sum(diag(tgammavar_acc_div))/2+sum(mu_acc_div))-(sqrt(sum(diag(tgammavar2_acc_div))/2+gt_acc_div%*%VarDeltaX_acc_div%*%t(gt_acc_div)))*dnorm(VaRstd_acc_div)/(pnorm(VaRstd_acc_div))

print("Total de acciones y divisas por CF")
print(VaR_acc_div_DGM_CF/sum(V0_acc_div))
print(CVaR_acc_div_DGM_CF/sum(V0_acc_div))

#Acciones
tgammavar3_acc=tgammavar2_acc%*%(tgammavar_acc)
asimP_acc=sum(diag(tgammavar3_acc))+3*t(gt_acc)%*%VarDeltaX_acc%*%gammat_acc%*%VarDeltaX_acc%*%(gt_acc) #gamma tercer momento
asimP_acc=asimP_acc/(sqrt(sum(diag(tgammavar2_acc))/2+t(gt_acc)%*%VarDeltaX_acc%*%(gt_acc)))^3  #coef de asimetría
VaRstd_acc=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_acc  #VaR estandarizado
VaR_acc_DGM_CF=(sum(diag(tgammavar_acc))/2+sum(mu_acc_div))+(sqrt(sum(diag(tgammavar2_acc))/2+t(gt_acc)%*%VarDeltaX_acc%*%(gt_acc)))*VaRstd_acc #VaR cf
CVaR_acc_DGM_CF=(sum(diag(tgammavar_acc))/2+sum(mu_acc_div))-(sqrt(sum(diag(tgammavar2_acc))/2+t(gt_acc)%*%VarDeltaX_acc%*%(gt_acc)))*dnorm(VaRstd_acc)/(pnorm(VaRstd_acc))

print("Total de acciones por CF")
print(VaR_acc_DGM_CF/sum(V0_acc))
print(CVaR_acc_DGM_CF/sum(V0_acc))

#Divisas
tgammavar3_div=tgammavar2_div%*%(tgammavar_div)
asimP_div=sum(diag(tgammavar3_div))+3*t(gt_div)%*%VarDeltaX_div%*%gammat_div%*%VarDeltaX_div%*%(gt_div) #gamma tercer momento
asimP_div=asimP_div/(sqrt(sum(diag(tgammavar2_div))/2+t(gt_div)%*%VarDeltaX_div%*%(gt_div)))^3  #coef de asimetría
VaRstd_div=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_div  #VaR estandarizado
VaR_div_DGM_CF=(sum(diag(tgammavar_div))/2+sum(mu_acc_div))+(sqrt(sum(diag(tgammavar2_div))/2+t(gt_div)%*%VarDeltaX_div%*%(gt_div)))*VaRstd_div #VaR cf
CVaR_div_DGM_CF=(sum(diag(tgammavar_div))/2+sum(mu_acc_div))-(sqrt(sum(diag(tgammavar2_div))/2+t(gt_div)%*%VarDeltaX_div%*%(gt_div)))*dnorm(VaRstd_div)/(pnorm(VaRstd_div))

print("Total de divisas por CF")
print(VaR_div_DGM_CF/sum(V0_div))
print(CVaR_div_DGM_CF/sum(V0_div))


[1] "VaR y CVaR por divisa"
          [,1]      [,2]      [,3]
[1,] -1.550658 -1.500521 -2.523029
          [,1]      [,2]     [,3]
[1,] -1.944588 -1.881714 -3.16398
[1] "VaR y CVaR por acc"
           [,1]        [,2]      [,3]
[1,] -0.2605417 -0.07714473 -2.840187
           [,1]       [,2]      [,3]
[1,] -0.3267298 -0.0967426 -3.561709
[1] "Total acciones y divisas"
            2010-03-30
2010-03-30 -0.01997689
            2010-03-30
2010-03-30 -0.02505183
[1] "Total acciones"
          [,1]
[1,] -2.908673
          [,1]
[1,] -3.647594
[1] "Total divisas"
          [,1]
[1,] -2.589685
         [,1]
[1,] -3.24757
[1] "Total de acciones y divisas por CF"
            2010-03-30
2010-03-30 -0.01997689
            2010-03-30
2010-03-30 -0.02505183
[1] "Total de acciones por CF"
          [,1]
[1,] -2.908673
          [,1]
[1,] -3.647594
[1] "Total de divisas por CF"
          [,1]
[1,] -2.589685
         [,1]
[1,] -3.24757


Podemos observar que en las divisas el VaR es mayor pasa USD/MXN, esto podría deberse a que hubo una caída del peso sobre el dolor justo en las fechas de valuación.

Como en las divisas el CVaR está más alejado del VaR, podemos decir que existen valores de pérdida más grandes al VaR a los que podríamos llegar, con un nivel de confianza del 95%. En cambio, en las acciones, vemos que estos valores no están tan alejados.

Ahora, para el total de acciones y el total de divisas, tienen un comportamiento muy similar entre ellos, la distancia entre VaR y CVaR es mayor para acciones que para divisas, por lo que hay un mayor riesgo de pérdida en acciones.

Sólo se obtiene Delta Normal, ya que para Delta Gamma y Cornish Fisher debemos de ocupar la segunda derivada, pero para acciones y divisas no tenemos segunda derivada, por lo que estos valores serán iguales para los 3 casos.

## RIESGO DE CETES

In [0]:
%%R


#Cálculo de matriz de pérdidas y ganancias CETES

#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_ftdc=DeltaX_port[,(1:n_if[1])] 
m_ftdc=ncol(plazos_bcc)
m=ncol(plazos_bcc)
VaRmargc_ftdc=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcstd_ftdc=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_ftdc=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_ftdc=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_ftdc=matrix(0,1,m) #CVar por contrato estandarizado


#mu_bd
#mu_bcc=bcc_dt(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc #primera derivada con respecto al tiempo evaluada en x0
#g_bcc=bcc_di(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc #gradiente evaluado en x0
#g_bcct=g_bcc*x0_bcc #gradienete evaluado en x0 por x0
#gamma_bcc=bcc_di2(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc #evaluación de gamma
#gamma_bcct2=diagv(gamma_bcc*(x0_bcc)^2) #evaluación de gamma multiplicado por x^{2}


#RIESGO TOTAL CETE

VarDeltaX_ftdc=var(DeltaX_ftdc)*(n/(n-1))

tgammavar_ftdc=(gamma_bcct2%*%VarDeltaX_ftdc)
tgammavar2_ftdc=(tgammavar_ftdc)%*%(tgammavar_ftdc) 
mudg_ftdc=sum(diag(tgammavar_ftdc))/2+sum(mu_bcc)
desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+g_bcct%*%VarDeltaX_ftdc%*%t(g_bcct))

#VaR Delta Normal

desv = sqrt(g_bcct%*%VarDeltaX_ftdc%*%t(g_bcct))
VaRT_ftdc_DN = mu_bcc - (desv*qnorm(alpha))
CVaRT_ftdc_DN = mu_bcc - (desv)*dnorm(qnorm(alpha))/(1-alpha)


#VaR Delta Gamma Normal
VaRT_ftdc_DGM = (mudg_ftdc) + (desv_ftdc)*qnorm(1-alpha)  
CVaRT_ftdc_DGM = mudg_ftdc - (desv_ftdc)*dnorm(qnorm(1-alpha))/(1-alpha)




###VaR Cornish Fisher
tgammavar3_ftdc=tgammavar2_ftdc%*%(tgammavar_ftdc)
asimP_ftdc=sum(diag(tgammavar3_ftdc))+3*g_bcct%*%VarDeltaX_ftdc%*%gamma_bcct2%*%VarDeltaX_ftdc%*%t(g_bcct) #gamma tercer momento
asimP_ftdc=asimP_ftdc/(desv_ftdc)^3  #coef de asimetría
VaRstd_ftdc=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_ftdc  #VaR estandarizado
VaRT_ftdc_DGM_CF=(mudg_ftdc)+(desv_ftdc)*VaRstd_ftdc #VaR cf
CVaRT_ftdc_DGM_CF= mudg_ftdc -(desv_ftdc)*dnorm(VaRstd_ftdc)/(pnorm(VaRstd_ftdc))

print("VaR TOTAL CETE DELTA NORMAL")
print(VaRT_ftdc_DN)
print("CVaR TOTAL CETE DELTA NORMAL")
print(CVaRT_ftdc_DN)
print("VaR TOTAL CETE DELTA GAMMA NORMAL")
print(VaRT_ftdc_DGM)
print("CVaR TOTAL CETE DELTA GAMMA NORMAL")
print(CVaRT_ftdc_DGM)
print("VaR TOTAL CETE CORNISH FISHER")
print(VaRT_ftdc_DGM_CF)
print("CVaR TOTAL CETE CORNISH FISHER")
print(CVaRT_ftdc_DGM_CF)

[1] "VaR TOTAL CETE DELTA NORMAL"
          [,1]
[1,] -9.491891
[1] "CVaR TOTAL CETE DELTA NORMAL"
          [,1]
[1,] -11.11148
[1] "VaR TOTAL CETE DELTA GAMMA NORMAL"
          [,1]
[1,] -9.490849
[1] "CVaR TOTAL CETE DELTA GAMMA NORMAL"
          [,1]
[1,] -11.11044
[1] "VaR TOTAL CETE CORNISH FISHER"
          [,1]
[1,] -9.489072
[1] "CVaR TOTAL CETE CORNISH FISHER"
          [,1]
[1,] -11.10891


Aqui podemos ver que es un valor pequeño esto se debe a que solo es un cete, entonces no hay mucha volatilidad, ademas comparando los 3 métodos podemos notar que son casi exactos entre ellos pues solo difieren en algunas milesimas ó son iguales

## RIESGO BONO M

In [0]:
%%R


#Cálculo de matriz de pérdidas y ganancias BONO M

#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_bm=DeltaX_port[,(1:n_if[2])] 
m_bm=ncol(plazos_bm)
m=ncol(plazos_bm)
VaRmargc_bm=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcstd_bm=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_bm=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_fbm=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_bm=matrix(0,1,m) #CVar por contrato estandarizado


#RIESGO TOTAL BONO M

VarDeltaX_bm=var(DeltaX_bm)*(n/(n-1))

tgammavar_bm=(gamma_bmt2%*%VarDeltaX_bm)
tgammavar2_bm=(tgammavar_bm)%*%(tgammavar_bm) 
mudg_ftdc=sum(diag(tgammavar_bm))/2+sum(mu_bm)
desv_ftdc=sqrt(sum(diag(tgammavar2_bm))/2+g_bmt%*%VarDeltaX_bm%*%t(g_bmt))

#VaR Delta Normal
desv_bonm = sqrt(g_bmt%*%VarDeltaX_bm%*%t(g_bmt))
VaRT_ftdc_DN = mu_bm - desv_bonm*qnorm(alpha)
CVaRT_ftdc_DN = mu_bm - desv_bonm*dnorm(qnorm(alpha))/(1-alpha)

#VaR Delta Gamma Normal
VaRT_ftdc_DGM = mudg_ftdc + (desv_ftdc)*qnorm(1-alpha)  
CVaRT_ftdc_DGM= mudg_ftdc - (desv_ftdc)*dnorm(qnorm(1-alpha))/(1-alpha)

###VaR Cornish Fisher
tgammavar3_ftdc = tgammavar2_bm%*%(tgammavar_bm)
asimP_ftdc = sum(diag(tgammavar3_ftdc))+3*g_bmt%*%VarDeltaX_ftdc%*%gamma_bcct2%*%VarDeltaX_ftdc%*%t(g_bmt) #gamma tercer momento
asimP_ftdc = asimP_ftdc/(desv_ftdc)^3  #coef de asimetría
VaRstd_ftdc = qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_ftdc  #VaR estandarizado
VaRT_ftdc_DGM_CF = (mudg_ftdc)+(desv_ftdc)*VaRstd_ftdc #VaR cf
CVaRT_ftdc_DGM_CF = mudg_ftdc -(desv_ftdc)*dnorm(VaRstd_ftdc)/(pnorm(VaRstd_ftdc))

print("VaR TOTAL BONO M DELTA NORMAL")
print(VaRT_ftdc_DN)
print("CVaR TOTAL BONO M DELTA NORMAL")
print(CVaRT_ftdc_DGM)
print("VaR TOTAL BONO M DELTA GAMMA NORMAL")
print(VaRT_ftdc_DGM)
print("CVaR TOTAL BONO M DELTA GAMMA NORMAL")
print(CVaRT_ftdc_DGM)
print("VaR TOTAL BONO M DELTA CORNISH-FISHER")
print(VaRT_ftdc_DGM_CF)
print("CVaR TOTAL BONO M DELTA CORNISH-FISHER")
print(CVaRT_ftdc_DGM_CF)

[1] "VaR TOTAL BONO M DELTA NORMAL"
          [,1]
[1,] -520.3674
[1] "CVaR TOTAL BONO M DELTA NORMAL"
          [,1]
[1,] -533.8582
[1] "VaR TOTAL BONO M DELTA GAMMA NORMAL"
          [,1]
[1,] -520.3679
[1] "CVaR TOTAL BONO M DELTA GAMMA NORMAL"
          [,1]
[1,] -533.8582
[1] "VaR TOTAL BONO M DELTA CORNISH-FISHER"
          [,1]
[1,] -520.3662
[1] "CVaR TOTAL BONO M DELTA CORNISH-FISHER"
          [,1]
[1,] -533.8567


En bonos M podemos notar que son valores un tanto pequeños pues al ser varios bonos y tener cupones no se tiene volatilidad, realmente este VaR deberia ser cero pero aqui se muestra como un tipo ejemplo de lo que podria pasar si no lo llegará a ser, aqui notamos que en valores absolutos estan por 400 y tantos 

## RIESGO BONDES D



### Riesgo a nivel total y total factor de riesgo

In [0]:
%%R
##### VaR TOTAL BONDE  #####

#VaR Normal
VaRTotalDN = mu_bd - sqrt(g_bdt%*%VarCovX_bd_ext%*%t(g_bdt))*qnorm(alpha) 
CVaRTotalDN = mu_bd - sqrt(g_bdt%*%VarCovX_bd_ext%*%t(g_bdt))*dnorm(qnorm(alpha))/(1-alpha)

#VaR Delta Gamma Normal
VaRTotalDGM = (sum(diag(tgammavar_bdt))/2+sum(mu_bd))+(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*qnorm(1-alpha) 
CVaRTotalDGM = (sum(diag(tgammavar_bdt))/2+sum(mu_bd))-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(qnorm(1-alpha))/(1-alpha)

###VaR Cornish Fisher
tgammavar3_bdt = tgammavar2_bdt%*%(gamma_bdt2%*%VarCovX_bd_ext) 
asimPmargc_bdt = sum(diag(tgammavar3_bdt))+3*g_bdt%*%VarCovX_bd_ext%*%gamma_bdt2%*%VarCovX_bd_ext%*%t(g_bdt) #gamma tercer momento
asimPmargc_bdt = asimPmargc_bdt/(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))^3  #coef de asimetría
VaRmargcstd_bdt = qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bdt  #VaR estandarizado
VaRTotalCF = (sum(diag(tgammavar_bdt))/2+sum(mu_bd))+(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*VaRmargcstd_bdt #VaR Normal #VaR cf
CVaRTotalCF = (sum(diag(tgammavar_bdt))/2+sum(mu_bd))-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(VaRmargcstd_bdt)/(pnorm(VaRmargcstd_bdt))

                                             ####   Por factor de riesgo ####
VaRTotalDNf=matrix(0,1,3)
CVaRTotalDNf=matrix(0,1,3)
VaRTotalDGMf=matrix(0,1,3)
CVaRTotalDGMf=matrix(0,1,3)
VaRTotalCFf=matrix(0,1,3)
CVaRTotalCFf=matrix(0,1,3)
 
 #MATRIZ de factores de riesgo primera fila cupón, segunda valor presente, tercera sobretasa
x0_bd_extf=rbind(cbind(matrix(tf_act,1,ncol(contratos_bdmT)),t(as.matrix(Xvp_bd[1,]))*0,t(as.matrix(Xst_bd[1,]))*0),
                 cbind(matrix(tf_act,1,ncol(contratos_bdmT))*0,t(as.matrix(Xvp_bd[1,])),t(as.matrix(Xst_bd[1,]))*0),
                 cbind(matrix(tf_act,1,ncol(contratos_bdmT))*0,t(as.matrix(Xvp_bd[1,]))*0,t(as.matrix(Xst_bd[1,]))))

for (j in (1:3))
{
g_bdt=g_bd*x0_bd_extf[j,]
gamma_bdt2=gamma_bd*((x0_bd_extf[j,])%*%t(x0_bd_extf[j,]))
VarCovX_bd_ext=cov(Delta_X_bd_ext)*(n-1)/(n-2)
tgammavar_bdt=(gamma_bdt2%*%VarCovX_bd_ext)
tgammavar2_bdt=(gamma_bdt2%*%VarCovX_bd_ext)%*%(gamma_bdt2%*%VarCovX_bd_ext) 

#var normal por riesgo
VaRTotalDNf[,j] = mu_bd - sqrt(g_bdt%*%VarCovX_bd_ext%*%t(g_bdt))*qnorm(alpha) 
CVaRTotalDNf[,j]= mu_bd - sqrt(g_bdt%*%VarCovX_bd_ext%*%t(g_bdt))*dnorm(qnorm(alpha))/(1-alpha)

#var delta gamma normal por riesgo
VaRTotalDGMf[,j] = (sum(diag(tgammavar_bdt))/2+sum(mu_bd)) + (sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*qnorm(1-alpha) #VaR Normal
CVaRTotalDGMf[,j] = (sum(diag(tgammavar_bdt))/2+sum(mu_bd)) - (sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(qnorm(1-alpha))/(1-alpha)

###VaR Cornish Fisher
tgammavar3_bdt = tgammavar2_bdt%*%(gamma_bdt2%*%VarCovX_bd_ext) #elevamos al cubo para sacar el tercer momento
asimPmargc_bdt = sum(diag(tgammavar3_bdt))+3*g_bdt%*%VarCovX_bd_ext%*%gamma_bdt2%*%VarCovX_bd_ext%*%t(g_bdt) #gamma tercer momento
asimPmargc_bdt = asimPmargc_bdt/(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))^3  #coef de asimetría
VaRmargcstd_bdt = qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bdt  #VaR estandarizado
VaRTotalCFf[,j] = (sum(diag(tgammavar_bdt))/2+sum(mu_bd)) + (sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*VaRmargcstd_bdt #VaR Normal #VaR cf
CVaRTotalCFf[,j] = (sum(diag(tgammavar_bdt))/2+sum(mu_bd)) - (sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(VaRmargcstd_bdt)/(pnorm(VaRmargcstd_bdt))
}


                                   
print("VaR TOTAL DEL BONDE DELTA NORMAL") 
print(VaRTotalDN)
print("CVaR TOTAL DEL BONDE DELTA NORMAL") 
print(CVaRTotalDN)
print("VaR TOTAL DEL BONDE DELTA GAMMA NORMAL") 
print(VaRTotalDGM)
print("CVaR TOTAL DEL BONDE DELTA GAMMA NORMAL") 
print(CVaRTotalDGM)
print("VaR TOTAL DEL BONDE CORNISH FISHER") 
print(VaRTotalCF) 
print("CVaR TOTAL DEL BONDE CORNISH FISHER")
print(CVaRTotalCF)
print("VaR TOTAL POR RIESGO DELTA NORMAL")
print(VaRTotalDNf)
print("CVaR TOTAL POR RIESGO DELTA NORMAL")
print(CVaRTotalDNf)
print("VaR TOTAL POR RIESGO DELTA GAMMA NORMAL")
print(VaRTotalDGMf)
print("CVaR TOTAL POR RIESGO DELTA GAMMA NORMAL")
print(CVaRTotalDGMf)
print("VaR TOTAL POR RIESGO CORNISH FISHER")
print(VaRTotalCFf)
print("CVaR TOTAL POR RIESGO CORNISH FISHER")
print(CVaRTotalCFf)


[1] "VaR TOTAL DEL BONDE DELTA NORMAL"
         [,1]
[1,] -297.099
[1] "CVaR TOTAL DEL BONDE DELTA NORMAL"
          [,1]
[1,] -366.4046
[1] "VaR TOTAL DEL BONDE DELTA GAMMA NORMAL"
          [,1]
[1,] -296.7963
[1] "CVaR TOTAL DEL BONDE DELTA GAMMA NORMAL"
          [,1]
[1,] -366.1022
[1] "VaR TOTAL DEL BONDE CORNISH FISHER"
          [,1]
[1,] -296.3148
[1] "CVaR TOTAL DEL BONDE CORNISH FISHER"
          [,1]
[1,] -365.6872
[1] "VaR TOTAL POR RIESGO DELTA NORMAL"
          [,1]     [,2]      [,3]
[1,] -83.49192 -220.452 -95.17417
[1] "CVaR TOTAL POR RIESGO DELTA NORMAL"
          [,1]      [,2]      [,3]
[1,] -98.53276 -270.2863 -113.1828
[1] "VaR TOTAL POR RIESGO DELTA GAMMA NORMAL"
          [,1]      [,2]      [,3]
[1,] -83.49192 -220.2882 -95.15275
[1] "CVaR TOTAL POR RIESGO DELTA GAMMA NORMAL"
          [,1]      [,2]      [,3]
[1,] -98.53276 -270.1226 -113.1614
[1] "VaR TOTAL POR RIESGO CORNISH FISHER"
          [,1]      [,2]      [,3]
[1,] -83.49192 -220.0108 -95.11647
[1] "

En este chunk tenemos el riesgo de Bondes a nivel total y total de factor de riesgo, si podemos ver en los primeros VaR y CVaR, varian de 80 y 100 en VaR y CVaR respectivamente de total del bono, ahora bien el valor de estos se va repartiendo es por eso que al hacer el total nos sale por ejemplo más chico y al hacerlo por riesgo estos son cada un más grande, se debe a la diversificacion que hay cuando es el total

###Riesgo a nivel contrato y contrato-factor de riesgo

In [0]:
%%R  
     #######   POR INSTRUMENTO   #########

#Para obtener el VaR por contrato
VaRContDN_bd=matrix(0,1,count(N_bd))
CVaRContDN_bd=matrix(0,1,count(N_bd))
VaRContDGM_bd=matrix(0,1,count(N_bd))
CVaRContDGM_bd=matrix(0,1,count(N_bd))
VaRContCF_bd=matrix(0,1,count(N_bd))
CVaRContCF_bd=matrix(0,1,count(N_bd))


                               ####     VaR por contrato por factor de riesgo   ####

VaRContDN_bdf=matrix(0,3,count(N_bd))
CVaRContDN_bdf=matrix(0,3,count(N_bd))
VaRContDGM_bdf=matrix(0,3,count(N_bd))
CVaRContDGM_bdf=matrix(0,3,count(N_bd))
VaRContCF_bdf=matrix(0,3,count(N_bd))
CVaRContCF_bdf=matrix(0,3,count(N_bd))

#V0=swap(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT, plazocuponT, VTplazos, Xvp[1,],N)

#Valor de las derivadas evaluadas en el punto x0.
V0f_dtc_bd=d_bondeD_tc(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
V0f_dvp_bd=d_bondeD_x(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
V0f_dst_bd=V0f_dvp_bd

for (j in (1:count(N_bd)))
{
  if(j==1)
  { 
    x0cont_bd=c(X_bd_tc[1,1:sum(N_bd[1:j])],Xvp_bd[1,1:sum(N_bd[1:j])],Xst_bd[1,1:sum(N_bd[1:j])])

    #MATRIZ de factores de riesgo primera fila cupón, segunda valor presente, tercera sobretasa
    x0cont_bdf=rbind(c(X_bd_tc[1,1:sum(N_bd[1:j])],Xvp_bd[1,1:sum(N_bd[1:j])]*0,Xst_bd[1,1:sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,1:sum(N_bd[1:j])]*0,Xvp_bd[1,1:sum(N_bd[1:j])],Xst_bd[1,1:sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,1:sum(N_bd[1:j])]*0,Xvp_bd[1,1:sum(N_bd[1:j])]*0,Xst_bd[1,1:sum(N_bd[1:j])]))
    mucont_bd=sum(mu_bd[j])
    gcont_bd=as.matrix(c(V0f_dtc_bd[1:sum(N_bd[1:j])],V0f_dvp_bd[1:sum(N_bd[1:j])],V0f_dst_bd[1:sum(N_bd[1:j])]))
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bd))
    VarCovXcont_bd=cov(cbind(X_bd_tc[,1:sum(N_bd[1:j])],Xvp_bd[,1:sum(N_bd[1:j])],Xst_bd[,1:sum(N_bd[1:j])]))*(n-1)/(n-2)	
    gamma_cont_bd=gamma_bd[1:(3*sum(N_bd[1:j])),1:(3*sum(N_bd[1:j]))]
    tgamma_bd= x0cont_bd%*%t(x0cont_bd)*gamma_cont_bd #gamma tilde
  }
  else
  {
    x0cont_bd=c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])])
    x0cont_bdf=rbind(c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))
    mucont_bd=sum(mu_bd[j])
    gcont_bd=as.matrix(c(V0f_dtc_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])],V0f_dvp_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])],V0f_dst_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bd))
    VarCovXcont_bd=cov(cbind(X_bd_tc[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xvp_bd[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xst_bd[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))*(n-1)/(n-2)
    gamma_cont_bd=gamma_bd[(3*sum(N_bd[1:j-1])+1):(3*sum(N_bd[1:j])),(3*sum(N_bd[1:j-1])+1):(3*sum(N_bd[1:j]))]
    tgamma_bd= x0cont_bd%*%t(x0cont_bd)*gamma_cont_bd #gamma tilde
  }
  
  #VaR Normal
  tgammavar_bd=(tgamma_bd%*%VarCovXcont_bd)
  tgammavar2_bd=(tgamma_bd%*%VarCovXcont_bd)%*%(tgamma_bd%*%VarCovXcont_bd)
  VaRContDN_bd[j] = mucont_bd - sqrt(gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd))*qnorm(alpha) 
  CVaRContDN_bd[j] = mucont_bd - sqrt(gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd))*dnorm(qnorm(alpha))/(1-alpha)
 
  #VaR Delta Gamma Normal
  VaRContDGM_bd[j] = (sum(diag(tgammavar_bd))/2+mucont_bd) + (sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*qnorm(1-alpha) 
  CVaRContDGM_bd[j] = (sum(diag(tgammavar_bd))/2+mucont_bd) - (sqrt(sum(diag(tgammavar_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(qnorm(1-alpha))/(1-alpha)
  
  ###VaR Cornish Fisher
  tgammavar3_bd=tgammavar2_bd%*%(tgamma_bd%*%VarCovXcont_bd)
  asimPmargc_bd=sum(diag(tgammavar3_bd))+3*gtcont_bd%*%VarCovXcont_bd%*%tgamma_bd%*%VarCovXcont_bd%*%t(gtcont_bd) #gamma tercer momento
  asimPmargc_bd=asimPmargc_bd/(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))^3  #coef de asimetría
  VaRmargcstd_bd=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bd  #VaR estandarizado
  VaRContCF_bd[j]=(sum(diag(tgammavar_bd))/2+mucont_bd)+(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*VaRmargcstd_bd #VaR Normal
  CVaRContCF_bd[j]=(sum(diag(tgammavar_bd))/2+sum(mucont_bd))-(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(VaRmargcstd_bd)/(pnorm(VaRmargcstd_bd))
  for (k in (1:3))
  {
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bdf[k,]))
    tgamma_bd= x0cont_bdf[k,]%*%t(x0cont_bdf[k,])*gamma_cont_bd #gamma tilde
    tgammavar_bd=(tgamma_bd%*%VarCovXcont_bd)
    tgammavar2_bd=(tgamma_bd%*%VarCovXcont_bd)%*%(tgamma_bd%*%VarCovXcont_bd)
    #VaR Delta Normal
    VaRContDN_bdf[k,j] = mucont_bd - sqrt(gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd))*qnorm(alpha) 
    CVaRContDN_bdf[k,j] = mucont_bd - sqrt(gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd))*dnorm(qnorm(alpha))/(1-alpha)
    #VaR Delta Gamma Normal
    VaRContDGM_bdf[k,j] = (sum(diag(tgammavar_bd))/2+mucont_bd) + (sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*qnorm(1-alpha) 
    CVaRContDGM_bdf[k,j] = (sum(diag(tgammavar_bd))/2+mucont_bd) - (sqrt(sum(diag(tgammavar_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(qnorm(1-alpha))/(1-alpha)
    ###VaR Cornish Fisher
    tgammavar3_bd = tgammavar2_bd%*%(tgamma_bd%*%VarCovXcont_bd)
    asimPmargc_bd = sum(diag(tgammavar3_bd))+3*gtcont_bd%*%VarCovXcont_bd%*%tgamma_bd%*%VarCovXcont_bd%*%t(gtcont_bd) #gamma tercer momento
    asimPmargc_bd = asimPmargc_bd/(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))^3  #coef de asimetría
    VaRmargcstd_bd = qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bd  #VaR estandarizado
    VaRContCF_bdf[k,j] = (sum(diag(tgammavar_bd))/2+mucont_bd)+(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*VaRmargcstd_bd #VaR Normal
    CVaRContCF_bdf[k,j] = (sum(diag(tgammavar_bd))/2+sum(mucont_bd)) - (sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(VaRmargcstd_bd)/(pnorm(VaRmargcstd_bd))    
  }
  
}
print("VaR POR CONTRATO DELTA NORMAL")
print(VaRContDN_bd)
print("CVaR POR CONTRATO DELTA NORMAL")
print(CVaRContDN_bd)
print("VaR POR CONTRATO DELTA GAMMA NORMAL")
print(VaRContDGM_bd)
print("CVaR POR CONTRATO DELTA GAMMA NORMAL")
print(CVaRContDGM_bd)
print("VaR POR CONTRATO CORNISH FISHER")
print(VaRContCF_bd)
print("CVaR POR CONTRATO CORNISH FISHER")
print(CVaRContCF_bd)
print("VaR POR CONTRATO POR FACTOR DE RIESGO DELTA NORMAL")
print(VaRContDN_bdf)
print("CVaR POR CONTRATO POR FACTOR DE RIESGO DELTA NORMAL")
print(CVaRContDN_bdf)
print("VaR POR CONTRATO POR FACTOR DE RIESGO DELTA GAMMA NORMAL")
print(VaRContDGM_bdf)
print("CVaR POR CONTRATO POR FACTOR DE RIESGO DELTA GAMMA NORMAL")
print(CVaRContDGM_bdf)
print("VaR POR CONTRATO POR FACTOR DE RIESGO CORNISH FISHER")
print(VaRContCF_bdf)
print("CVaR POR CONTRATO POR FACTOR DE RIESGO CORNISH FISHER")
print(CVaRContCF_bdf)

[1] "VaR POR CONTRATO DELTA NORMAL"
          [,1]
[1,] -172.1384
[1] "CVaR POR CONTRATO DELTA NORMAL"
         [,1]
[1,] -209.699
[1] "VaR POR CONTRATO DELTA GAMMA NORMAL"
          [,1]
[1,] -172.0414
[1] "CVaR POR CONTRATO DELTA GAMMA NORMAL"
         [,1]
[1,] -209.603
[1] "VaR POR CONTRATO CORNISH FISHER"
          [,1]
[1,] -171.8753
[1] "CVaR POR CONTRATO CORNISH FISHER"
         [,1]
[1,] -209.459
[1] "VaR POR CONTRATO POR FACTOR DE RIESGO DELTA NORMAL"
           [,1]
[1,]  -33.67358
[2,] -161.97474
[3,]  -42.22189
[1] "CVaR POR CONTRATO POR FACTOR DE RIESGO DELTA NORMAL"
           [,1]
[1,]  -36.05855
[2,] -196.95340
[3,]  -46.77849
[1] "VaR POR CONTRATO POR FACTOR DE RIESGO DELTA GAMMA NORMAL"
           [,1]
[1,]  -33.67358
[2,] -161.89493
[3,]  -42.22053
[1] "CVaR POR CONTRATO POR FACTOR DE RIESGO DELTA GAMMA NORMAL"
           [,1]
[1,]  -36.05855
[2,] -196.87444
[3,]  -46.77726
[1] "VaR POR CONTRATO POR FACTOR DE RIESGO CORNISH FISHER"
           [,1]
[1,]  -33.67358
[2

En este se puede observar el riesgo que existe a nivel contrato y contrato por factor de riesgo, aqui pasa algo diferente al total y total por riesgo es mas grande en valor absoluto el VaR y CVaR de un contrato a que si lo ves por contrato y factor de riesgo, y estos a asu vez son más pequeños a el total.

## RIESGO FORWARDS TDC

### Riesgo a nivel contrato y nivel contrato-factor de riesgo

In [0]:
%%R


#Cálculo de matriz de pérdidas y ganancias FUTUROS TDC

#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_ftdc=DeltaX_port_forwards[,(1:n_if_forwards[1])] 
m_ftdc=ncol(plazos_fwd)
m=ncol(plazos_fwd)
VaRmargc_ftdc=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcstd_ftdc=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_ftdc=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_ftdc=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_ftdc=matrix(0,1,m) #CVar por contrato estandarizado
VaRxcont_ftdc= matrix(0,1,m) # Normal
CVaRxcont_ftdc= matrix(0,1,m)

##factores riesgo
#Spot
VaRmargc_fr_ftdc=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargcstd_fr_ftdc=matrix(0,3,m) #Var por contrato estandarizado 
VaRmargcCF_fr_ftdc=matrix(0,3,m) #Var por contrato Cornish-Fisher
CVaRmargc_fr_ftdc=matrix(0,3,m) #CVar por contrato estandarizado
CVaRmargcCF_fr_ftdc=matrix(0,3,m) #CVar por contrato estandarizado
VaRnormal_fr=matrix(0,3,m)#vaR Normal
CVaRnormal_fr=matrix(0,3,m)#vaR Normal
WFR=diagv(c(1,1,1))


for (i in 1:(m))
{
  delta_ftdc=t(g_ftdc[seq(i,m*3,by=m)]) 
  tdelta_ftdc=t(g_ftdc[seq(i,m*3,by=m)])*xp_ftdc[seq(i,m*3,by=m)]   #delta tilde
  gamma_ftdc=mgamma_futTC(plazos_fwd[i],X1_fwtdc[1,i],X2_fwtdc[1,i],X3_ftdc[1,],kst_fwd[i],contratos_fwd[i],nominal_fwd)  #paso clave
  tgamma_ftdc=xp_ftdc[seq(i,m*3,by=m)]%*%t(xp_ftdc[seq(i,m*3,by=m)])*gamma_ftdc #gamma tilde
  Varmargcaux_ftdc=cov(DeltaX_ftdc[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)
  tgammavar2_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)%*%(tgamma_ftdc%*%Varmargcaux_ftdc)  
  mudg_ftdc=sum(diag(tgammavar_ftdc))/2+mu_ftdc[i]
  desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))

  #var normal por contrato
VaRxcont_ftdc[i] = mudg_ftdc - sqrt(tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))*qnorm(alpha) 
CVaRxcont_ftdc[i]= mudg_ftdc - sqrt(tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))*dnorm(qnorm(alpha))/(1-alpha)

  VaRmargc_ftdc[i]=mudg_ftdc+desv_ftdc*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_ftdc[i]=-mudg_ftdc-desv_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_ftdc=tgammavar2_ftdc%*%(tgamma_ftdc%*%Varmargcaux_ftdc)
  asimPmargc_ftdc=sum(diag(tgammavar3_ftdc))+3*tdelta_ftdc%*%Varmargcaux_ftdc%*%tgamma_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc) #tercer momento
  asimPmargc_ftdc=asimPmargc_ftdc/(desv_ftdc)^3  #coef de asimetría
  VaRmargcstd_ftdc[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_ftdc  #VaR estandarizado
  VaRmargcCF_ftdc[i]=mudg_ftdc+desv_ftdc*VaRmargcstd_ftdc[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_ftdc[i]=-mudg_ftdc-desv_ftdc*(dnorm(VaRmargcstd_ftdc[i])/(pnorm(VaRmargcstd_ftdc[i])))

#Por factor de riesgo y por contrato
#spot, tasa extranjera, tasa nacional
  for (j in 1:3)
  {
    WFRM=matrix(0,3,3)  #PASO CLAVE
    WFRM[j,j]=1  #PASO CLAVE
    delta_ftdc=t(g_ftdc[seq(i,m*3,by=m)])*WFR[j,] #PASO CLAVE *(1,0,0), (0,1,0), (0,0,1)
    #print(WFR[j,])
    tdelta_ftdc=t(g_ftdc[seq(i,m*3,by=m)])*xp_ftdc[seq(i,m*3,by=m)]*WFR[j,]   #delta tilde #PASO CLAVE
    gamma_ftdc=mgamma_futTC(plazos_fwd[i],X1_fwtdc[1,i],X2_fwtdc[1,i],X3_ftdc[1,],kst_fwd[i],contratos_fwd[i],nominal_fwd)*WFRM  #paso clave
   #print(WFRM)
    tgamma_ftdc=xp_ftdc[seq(i,m*3,by=m)]%*%t(xp_ftdc[seq(i,m*3,by=m)])*gamma_ftdc #gamma tilde
    Varmargcaux_ftdc=cov(DeltaX_ftdc[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)
    tgammavar2_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)%*%(tgamma_ftdc%*%Varmargcaux_ftdc)  
    #mudg_ftdc=sum(diag(tgammavar_ftdc))/2+mu_ftdc[i]
    mudg_ftdc=0
    desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))
     
#var normal por riesgo
VaRnormal_fr[j,i] = mudg_ftdc - sqrt(tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))*qnorm(alpha) 
CVaRnormal_fr[j,i]= mudg_ftdc - sqrt(tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))*dnorm(qnorm(alpha))/(1-alpha)

    VaRmargc_fr_ftdc[j,i]=mudg_ftdc+desv_ftdc*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_ftdc[j,i]=-mudg_ftdc-desv_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))
    
  
    tgammavar3_ftdc=tgammavar2_ftdc%*%(tgamma_ftdc%*%Varmargcaux_ftdc)
    asimPmargc_ftdc=sum(diag(tgammavar3_ftdc))+3*tdelta_ftdc%*%Varmargcaux_ftdc%*%tgamma_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc) #tercer momento
    asimPmargc_ftdc=asimPmargc_ftdc/(desv_ftdc)^3  #coef de asimetría
    VaRmargcstd_fr_ftdc[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_ftdc  #VaR estandarizado
    VaRmargcCF_fr_ftdc[j,i]=mudg_ftdc+desv_ftdc*VaRmargcstd_fr_ftdc[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_ftdc[j,i]=-mudg_ftdc-desv_ftdc*(dnorm(VaRmargcstd_fr_ftdc[j,i])/(pnorm(VaRmargcstd_fr_ftdc[j,i])))
  }



}

print("Valor a nivel contrato")
print(V0_fwtdc)
print("Valor a nivel contrato normal")
print(VaRxcont_ftdc)
print("CValor a nivel contrato normal")
print(CVaRxcont_ftdc)
print("VaR a nivel contrato delta gamma normal")
print(VaRmargc_ftdc)
print("VaR a nivel contrato CF")
print(VaRmargcCF_ftdc)
print("CVaR a nivel contrato delta gamma normal")
print(CVaRmargc_ftdc)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_ftdc)

print("VaR normal a nivel contrato")
print(VaRnormal_fr)
print("CVaR normal a nivel contrato")
print(CVaRnormal_fr)
print("VaR a nivel contrato  delta gamma normal")
print(VaRmargc_fr_ftdc)
print("VaR a nivel contrato  CF")
print(VaRmargcCF_fr_ftdc)
print("CVaR a nivel contrato delta gamma normal")
print(CVaRmargc_fr_ftdc)
print("CVaR a nivel contrato  CF")
print(CVaRmargcCF_fr_ftdc)


[1] "Valor a nivel contrato"
          [,1]
[1,] -53.23399
[1] "Valor a nivel contrato normal"
          [,1]
[1,] -18.11104
[1] "CValor a nivel contrato normal"
          [,1]
[1,] -22.71501
[1] "VaR a nivel contrato delta gamma normal"
          [,1]
[1,] -18.11104
[1] "VaR a nivel contrato CF"
          [,1]
[1,] -18.11105
[1] "CVaR a nivel contrato delta gamma normal"
          [,1]
[1,] -22.73885
[1] "CVaR a nivel contrato CF"
          [,1]
[1,] -22.73886
[1] "VaR normal a nivel contrato"
             [,1]
[1,] -18.12238670
[2,]  -0.00689637
[3,]  -0.01437149
[1] "CVaR normal a nivel contrato"
              [,1]
[1,] -22.726204050
[2,]  -0.008648326
[3,]  -0.018022426
[1] "VaR a nivel contrato  delta gamma normal"
             [,1]
[1,] -18.12238670
[2,]  -0.00689637
[3,]  -0.01437149
[1] "VaR a nivel contrato  CF"
             [,1]
[1,] -18.12238670
[2,]  -0.00689637
[3,]  -0.01437149
[1] "CVaR a nivel contrato delta gamma normal"
              [,1]
[1,] -22.726204050
[2,]  -0.0

Aqui podemos ver el riesgo que nos da al ver los forwards tdc a nivel contrato y nuevamente notamos que entre los 3 métodos no hay mucha diferencia en realidad asi que son algo exactos entre si

###Riesgo a nivel total y total factor de riesgo

In [0]:
%%R


#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO

VarDeltaX_ftdc=cov(DeltaX_ftdc)*(n/(n-1))

tgammavar_ftdc=(gamma_ftdctT2%*%VarDeltaX_ftdc)
tgammavar2_ftdc=(tgammavar_ftdc)%*%(tgammavar_ftdc) 
mudg_ftdc=sum(diag(tgammavar_ftdc))/2+sum(mu_ftdc)
desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+g_ftdctT%*%VarDeltaX_ftdc%*%t(g_ftdctT))

VaRT_ftdc_DGM=(mudg_ftdc)+(desv_ftdc)*qnorm(1-alpha)  #VaR Normal
CVaRT_ftdc_DGM=-(mudg_ftdc)-(desv_ftdc)*dnorm(qnorm(alpha))/(1-alpha)

#VaR Normal
VaRTotalN_fwtdc = mu_ftdc- sqrt(g_ftdctT%*%VarDeltaX_ftdc%*%t(g_ftdctT))*qnorm(alpha) 
CVaRTotalN_fwtdc = -mu_ftdc - sqrt(g_ftdctT%*%VarDeltaX_ftdc%*%t(g_ftdctT))*dnorm(qnorm(alpha))/(1-alpha)

###VaR Cornish Fisher
tgammavar3_ftdc=tgammavar2_ftdc%*%(tgammavar_ftdc)
asimP_ftdc=sum(diag(tgammavar3_ftdc))+3*g_ftdctT%*%VarDeltaX_ftdc%*%gamma_ftdctT2%*%VarDeltaX_ftdc%*%t(g_ftdctT) #gamma tercer momento
asimP_ftdc=asimP_ftdc/(desv_ftdc)^3  #coef de asimetría
VaRstd_ftdc=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_ftdc  #VaR estandarizado
VaRT_ftdc_DGM_CF=(mudg_ftdc)+(desv_ftdc)*VaRstd_ftdc #VaR cf
CVaRT_ftdc_DGM_CF=-(mudg_ftdc)-(desv_ftdc)*dnorm(VaRstd_ftdc)/(pnorm(VaRstd_ftdc))

print("VAR Total de FWTDC por Normal")
print(VaRTotalN_fwtdc)
print("CVAR Total de FWTDC por Normal")
print(CVaRTotalN_fwtdc)
print("VAR Total de FWTDC por Delta Gamma Normal")
print(VaRT_ftdc_DGM)
print("CVAR Total de FWTDC por Delta Gamma Normal")
print(CVaRT_ftdc_DGM)
print("VAR Total de FWTDC por Delta Gamma CF")
print(VaRT_ftdc_DGM_CF)
print("CVAR Total de FWTDC por Delta Gamma CF")
print(CVaRT_ftdc_DGM_CF)

VaRT_fr_ftdc_DGM=matrix(0,1,3)
CVaRT_fr_ftdc_DGM=matrix(0,1,3)
VaRT_fr_ftdc_DGM_CF=matrix(0,1,3)
CVaRT_fr_ftdc_DGM_CF=matrix(0,1,3)
VaRTotal_ftdc=matrix(0,1,3)
CVaRTotal_ftdc=matrix(0,1,3)

WFRT=matrix(0,3,(m*3))
# WFRT[1,1:m]=matrix(1,1,m)
# WFRT[2,(m+1):(2*m)]=matrix(1,1,m)
# WFRT[3,(2*m+1):(3*m)]=matrix(1,1,m)
#print(WFRT)

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRM=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE    
gamma_ftdctT2_fr=gamma_ftdctT2*WFRM #evaluación de gamma multiplicado por x^{2}
g_ftdctT_fr=g_ftdctT*WFRT[j,]
# print(WFRM)
# print(g_ftdctT_fr)
# print(gamma_ftdctT2_fr)
# print(WFRT)
tgammavar_ftdc=(gamma_ftdctT2_fr%*%VarDeltaX_ftdc)
tgammavar2_ftdc=(tgammavar_ftdc)%*%(tgammavar_ftdc) 
#mudg_ftdc=sum(diag(tgammavar_ftdc))/2+sum(mu_ftdc)
mudg_ftdc=0
desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+g_ftdctT_fr%*%VarDeltaX_ftdc%*%t(g_ftdctT_fr))
VaRT_fr_ftdc_DGM[j]=(mudg_ftdc)+(desv_ftdc)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_ftdc_DGM[j]=-(mudg_ftdc)-(desv_ftdc)*dnorm(qnorm(alpha))/(1-alpha)

#var normal por riesgo
VaRTotal_ftdc[,j] = mu_ftdc - sqrt(g_ftdctT%*%VarDeltaX_ftdc%*%t(g_ftdctT))*qnorm(alpha) 
CVaRTotal_ftdc[,j]= mu_ftdc - sqrt(g_ftdctT%*%VarDeltaX_ftdc%*%t(g_ftdctT))*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_ftdc=tgammavar2_ftdc%*%(tgammavar_ftdc)
asimP_ftdc=sum(diag(tgammavar3_ftdc))+3*g_ftdctT_fr%*%VarDeltaX_ftdc%*%gamma_ftdctT2_fr%*%VarDeltaX_ftdc%*%t(g_ftdctT_fr) #gamma tercer momento
 #print(asimP_ftdc)
asimP_ftdc=asimP_ftdc/(desv_ftdc)^3  #coef de asimetría
 
VaRstd_ftdc=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_ftdc  #VaR estandarizado
#print(VaRstd_ftdc)
VaRT_fr_ftdc_DGM_CF[j]=(mudg_ftdc)+(desv_ftdc)*VaRstd_ftdc #VaR cf
CVaRT_fr_ftdc_DGM_CF[j]=-(mudg_ftdc)-(desv_ftdc)*dnorm(VaRstd_ftdc)/(pnorm(VaRstd_ftdc))


}

print("VaR por riesgo por Normal")
print(VaRTotal_ftdc)
print("CVaR por riesgo por Normal")
print(CVaRTotal_ftdc)
print("VaR por riesgo por Delta Gamma Normal")
print(VaRT_fr_ftdc_DGM)
print("CVaR por riesgo por Delta Gamma Normal")
print(CVaRT_fr_ftdc_DGM)
print("VaR por riesgo por Delta Gamma CF")
print(VaRT_fr_ftdc_DGM_CF)
print("CVaR por riesgo por Delta Gamma CF")
print(CVaRT_fr_ftdc_DGM_CF)




# cbind(VaRTotal_fwtdc,sum(V0_fwtdc), VaRCont_fwtdc, V0_fwtdc)
# cbind(CVaRTotal_fwtdc,sum(V0_fwtdc), CVaRCont_fwtdc, V0_fwtdc)
# cbind(VaRTotal_fwtdc,VaRTotalfr1_fwtdc,VaRTotalfr2_fwtdc,VaRTotalfr3_fwtdc)
# cbind(CVaRTotal_fwtdc,CVaRTotalfr1_fwtdc,CVaRTotalfr2_fwtdc,CVaRTotalfr3_fwtdc)


[1] "VAR Total de FWTDC por Normal"
          [,1]
[1,] -18.11104
[1] "CVAR Total de FWTDC por Normal"
          [,1]
[1,] -22.73885
[1] "VAR Total de FWTDC por Delta Gamma Normal"
          [,1]
[1,] -18.11104
[1] "CVAR Total de FWTDC por Delta Gamma Normal"
          [,1]
[1,] -22.73885
[1] "VAR Total de FWTDC por Delta Gamma CF"
          [,1]
[1,] -18.11105
[1] "CVAR Total de FWTDC por Delta Gamma CF"
          [,1]
[1,] -22.73886
[1] "VaR por riesgo por Normal"
          [,1]      [,2]      [,3]
[1,] -18.11104 -18.11104 -18.11104
[1] "CVaR por riesgo por Normal"
          [,1]      [,2]      [,3]
[1,] -22.71501 -22.71501 -22.71501
[1] "VaR por riesgo por Delta Gamma Normal"
          [,1]        [,2]        [,3]
[1,] -18.12239 -0.00689637 -0.01437149
[1] "CVaR por riesgo por Delta Gamma Normal"
         [,1]         [,2]        [,3]
[1,] -22.7262 -0.008648326 -0.01802243
[1] "VaR por riesgo por Delta Gamma CF"
          [,1]        [,2]        [,3]
[1,] -18.12239 -0.00689637 -0.01

el valor que vemos en valores absolutos son pequeños eso se debe a que ya es el total y suele pasar cuando se diversifica el portafolio, aunque el total por riesgo es un poco mas pequeño

##Riesgo Forwards Índices

###Riesgo a nivel contrato y contrato-factor de riesgo

In [0]:
%%R

DeltaX_find=DeltaX_port_forwards[,(n_if_forwards[1]+1):(sum(n_if_forwards[1:2]))] 
m=ncol(plazos_fwd_ind)
m_find=ncol(plazos_fwd_ind)
VaRmargc_find=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcstd_find=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_find=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_find=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_find=matrix(0,1,m) #CVar por contrato estandarizado
VaRxcont_find=matrix(0,1,m) #var normal
CVaRxcont_find=matrix(0,1,m) #var normal
##factores riesgo
#Spot
VaRmargc_fr_find=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargcstd_fr_find=matrix(0,3,m) #Var por contrato estandarizado 
VaRmargcCF_fr_find=matrix(0,3,m) #Var por contrato Cornish-Fisher
CVaRmargc_fr_find=matrix(0,3,m) #CVar por contrato estandarizado
CVaRmargcCF_fr_find=matrix(0,3,m) #CVar por contrato estandarizado
VaRContDN_fw=matrix(0,3,m) #VaR Normal
CVaRContDN_fw=matrix(0,3,m) #CVaR Normal
WFR=diagv(c(1,1,1))


for (i in 1:(m))
{
  delta_find=t(g_find[seq(i,m*3,by=m)]) 
  tdelta_find=t(g_find[seq(i,m*3,by=m)])*xp_find[seq(i,m*3,by=m)]   #delta tilde
  gamma_find=mgamma_futTC(plazos_fwd_ind[i],X1_fwind[1,i],X2_fwind[1,i],X3_find[1,],kst_fwd_ind[i],contratos_fwd_ind[i],nominal_fwd_ind)  #paso clave
  tgamma_find=xp_find[seq(i,m*3,by=m)]%*%t(xp_find[seq(i,m*3,by=m)])*gamma_find #gamma tilde
  Varmargcaux_find=cov(DeltaX_find[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_find=(tgamma_find%*%Varmargcaux_find)
  tgammavar2_find=(tgamma_find%*%Varmargcaux_find)%*%(tgamma_find%*%Varmargcaux_find)  
  mudg_find=sum(diag(tgammavar_find))/2+mu_find[i]
  #mudg_find=0
  desv_find=sqrt(sum(diag(tgammavar2_find))/2+tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))

#var normal por contrato
VaRxcont_find[i] = mudg_find - sqrt(tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))*qnorm(alpha) 
CVaRxcont_find[i]= mudg_find - sqrt(tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))*dnorm(qnorm(alpha))/(1-alpha)


  VaRmargc_find[i]=mudg_find+desv_find*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_find[i]=-mudg_find-desv_find*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_find=tgammavar2_find%*%(tgamma_find%*%Varmargcaux_find)
  asimPmargc_find=sum(diag(tgammavar3_find))+3*tdelta_find%*%Varmargcaux_find%*%tgamma_find%*%Varmargcaux_find%*%t(tdelta_find) #tercer momento
  asimPmargc_find=asimPmargc_find/(desv_find)^3  #coef de asimetría
  VaRmargcstd_find[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_find  #VaR estandarizado
  VaRmargcCF_find[i]=mudg_find+desv_find*VaRmargcstd_find[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_find[i]=-mudg_find-desv_find*(dnorm(VaRmargcstd_find[i])/(pnorm(VaRmargcstd_find[i])))

#Por factor de riesgo y por contrato

  for (j in 1:3)
  {
    WFRM=matrix(0,3,3)  #PASO CLAVE
    WFRM[j,j]=1  #PASO CLAVE
    delta_find=t(g_find[seq(i,m*3,by=m)])*WFR[j,] #PASO CLAVE
    tdelta_find=t(g_find[seq(i,m*3,by=m)])*xp_find[seq(i,m*3,by=m)]*WFR[j,]   #delta tilde #PASO CLAVE
  	gamma_find=mgamma_futTC(plazos_fwd_ind[i],X1_fwind[1,i],X2_fwind[1,i],X3_find[1,],kst_fwd_ind[i],contratos_fwd_ind[i],nominal_fwd_ind)  #paso clave
    tgamma_find=xp_find[seq(i,m*3,by=m)]%*%t(xp_find[seq(i,m*3,by=m)])*gamma_find #gamma tilde
    Varmargcaux_find=cov(DeltaX_find[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_find=(tgamma_find%*%Varmargcaux_find)
    tgammavar2_find=(tgamma_find%*%Varmargcaux_find)%*%(tgamma_find%*%Varmargcaux_find)  
    #mudg_find=sum(diag(tgammavar_find))/2+mu_find[i]
   mudg_find=0
    desv_find=sqrt(sum(diag(tgammavar2_find))/2+tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))
    # Normal
    VaRContDN_fw[j,i] = (mudg_find - sqrt(tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))*qnorm(alpha))/10000
  CVaRContDN_fw[j,i] = (mudg_find - sqrt(tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))*dnorm(qnorm(alpha))/(1-alpha))/10000
 
    VaRmargc_fr_find[j,i]=mudg_find+desv_find*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_find[j,i]=-mudg_find-desv_find*(dnorm(qnorm(1-alpha))/(1-alpha))
    
    tgammavar3_find=tgammavar2_find%*%(tgamma_find%*%Varmargcaux_find)
    asimPmargc_find=sum(diag(tgammavar3_find))+3*tdelta_find%*%Varmargcaux_find%*%tgamma_find%*%Varmargcaux_find%*%t(tdelta_find) #tercer momento
    asimPmargc_find=asimPmargc_find/(desv_find)^3  #coef de asimetría
    VaRmargcstd_fr_find[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_find  #VaR estandarizado
    VaRmargcCF_fr_find[j,i]=mudg_find+desv_find*VaRmargcstd_fr_find[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_find[j,i]=-mudg_find-desv_find*(dnorm(VaRmargcstd_fr_find[j,i])/(pnorm(VaRmargcstd_fr_find[j,i])))
  }



}


print("Valor a nivel contrato")
print(V0_fwind)

print("VaR a nivel contrato NORMAL")
print(VaRxcont_find)
print("CVaR a nivel contrato NORMAL")
print(CVaRxcont_find)
print("VaR a nivel contrato Delta Gamma Normal")
print(VaRmargc_find)
print("VaR a nivel contrato CF")
print(VaRmargcCF_find)
print("CVaR a nivel contrato Delta Gamma Normal")
print(CVaRmargc_find)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_find)

print("VaR a nivel contrato Normal")
print(VaRContDN_fw)
print("CVaR a nivel contrato Normal")
print(CVaRContDN_fw)
print("VaR a nivel contrato Delta Gamma Normal")
print(VaRmargc_fr_find)
print("VaR a nivel contrato CF")
print(VaRmargcCF_fr_find)
print("CVaR a nivel contrato Delta Gamma Normal")
print(CVaRmargc_fr_find)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_fr_find)

[1] "Valor a nivel contrato"
        [,1]
[1,] 4528398
[1] "VaR a nivel contrato NORMAL"
          [,1]
[1,] -32718.61
[1] "CVaR a nivel contrato NORMAL"
          [,1]
[1,] -40784.35
[1] "VaR a nivel contrato Delta Gamma Normal"
          [,1]
[1,] -32718.61
[1] "VaR a nivel contrato CF"
          [,1]
[1,] -32718.61
[1] "CVaR a nivel contrato Delta Gamma Normal"
          [,1]
[1,] -38846.83
[1] "CVaR a nivel contrato CF"
          [,1]
[1,] -38846.83
[1] "VaR a nivel contrato Normal"
           [,1]
[1,] -3.1726138
[2,]  0.0000000
[3,] -0.0383918
[1] "CVaR a nivel contrato Normal"
            [,1]
[1,] -3.97858571
[2,]  0.00000000
[3,] -0.04814486
[1] "VaR a nivel contrato Delta Gamma Normal"
              [,1]
[1,] -3.172614e+04
[2,] -9.575838e-02
[3,] -3.839180e+02
[1] "VaR a nivel contrato CF"
              [,1]
[1,] -3.172614e+04
[2,] -4.895189e-02
[3,] -3.838478e+02
[1] "CVaR a nivel contrato Delta Gamma Normal"
              [,1]
[1,] -3.978586e+04
[2,] -1.200849e-01
[3,] -4.8

al ser un forward de indices podemos notar que es mas grande en valores aboslutos a comparacion del de tipo de cambio esto puede ser por la volatilidad con la que se manejan los indices, ademas de ser muchisimo mas pequeños cuando se hace por contrato e indice

###Riesgo a nivel total y total factor de riesgo

In [0]:
%%R


#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO

VarDeltaX_find=cov(DeltaX_find)*(n/(n-1))

tgammavar_find=(gamma_findtT2%*%VarDeltaX_find)
tgammavar2_find=(tgammavar_find)%*%(tgammavar_find) 
#media
mudg_find=sum(diag(tgammavar_find))/2+sum(mu_find)
#desviación estándar
desv_find=sqrt(sum(diag(tgammavar2_find))/2+g_findtT%*%VarDeltaX_find%*%t(g_findtT))

#VaR Delta Gamma Normal
VaRT_find_DGM=(mudg_find)+(desv_find)*qnorm(1-alpha)  #VaR Normal
CVaRT_find_DGM=-(mudg_find)-(desv_find)*dnorm(qnorm(alpha))/(1-alpha)

#VaR Normal
VaRNormal_find = mu_find - sqrt(g_findtT%*%VarDeltaX_find%*%t(g_findtT))*qnorm(alpha) 
CVaRNormal_find = -mu_find - sqrt(g_findtT%*%VarDeltaX_find%*%t(g_findtT))*dnorm(qnorm(alpha))/(1-alpha)



###VaR Cornish Fisher
tgammavar3_find=tgammavar2_find%*%(tgammavar_find)
asimP_find=sum(diag(tgammavar3_find))+3*g_findtT%*%VarDeltaX_find%*%gamma_findtT2%*%VarDeltaX_find%*%t(g_findtT) #gamma tercer momento
asimP_find=asimP_find/(desv_find)^3  #coef de asimetría
VaRstd_find=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_find  #VaR estandarizado
VaRT_find_DGM_CF=(mudg_find)+(desv_find)*VaRstd_find #VaR cf
CVaRT_find_DGM_CF=-(mudg_find)-(desv_find)*dnorm(VaRstd_find)/(pnorm(VaRstd_find))

print("VaR Normal total de Forward Indices")
print(VaRNormal_find)
print("VaR Normal total de Forward Indices")
print(CVaRNormal_find)
print("VaR total de Forward Indices Delta Gamma Normal")
print(VaRT_find_DGM)
print("CVaR total de Forward Indices Delta Gamma Normal")
print(CVaRT_find_DGM)
print("VaR total de Forward Indices Delta Gamma CF")
print(VaRT_find_DGM_CF)
print("CVaR total de Forward Indices Delta Gamma CF")
print(CVaRT_find_DGM_CF)

VaRT_fr_find_DGM=matrix(0,1,3)
CVaRT_fr_find_DGM=matrix(0,1,3)
VaRT_fr_find_DGM_CF=matrix(0,1,3)
CVaRT_fr_find_DGM_CF=matrix(0,1,3)
VaRNormal_find1 =matrix(0,1,3)
CVaRNormal_find1 =matrix(0,1,3)
WFRT=matrix(0,3,(m*3))

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRM=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE    
gamma_findtT2_fr=gamma_findtT2*WFRM #evaluación de gamma multiplicado por x^{2}
g_findtT_fr=g_findtT*WFRT[j,]
tgammavar_find=(gamma_findtT2_fr%*%VarDeltaX_find)
tgammavar2_find=(tgammavar_find)%*%(tgammavar_find) 
#mudg_find=sum(diag(tgammavar_find))/2+sum(mu_find)
mudg_find=0
desv_find=sqrt(sum(diag(tgammavar2_find))/2+g_findtT_fr%*%VarDeltaX_find%*%t(g_findtT_fr))
VaRT_fr_find_DGM[j]=(mudg_find)+(desv_find)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_find_DGM[j]=-(mudg_find)-(desv_find)*dnorm(qnorm(alpha))/(1-alpha)

#VaR Normal
VaRNormal_find1[j] = mudg_find - sqrt(g_findtT_fr%*%VarDeltaX_find%*%t(g_findtT_fr))*qnorm(alpha) 
CVaRNormal_find1[j]= -mudg_find - sqrt(g_findtT_fr%*%VarDeltaX_find%*%t(g_findtT_fr))*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_find=tgammavar2_find%*%(tgammavar_find)
asimP_find=sum(diag(tgammavar3_find))+3*g_findtT_fr%*%VarDeltaX_find%*%gamma_findtT2_fr%*%VarDeltaX_find%*%t(g_findtT_fr) #gamma tercer momento
asimP_find=asimP_find/(desv_find)^3  #coef de asimetría
 
VaRstd_find=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_find  #VaR estandarizado
VaRT_fr_find_DGM_CF[j]=(mudg_find)+(desv_find)*VaRstd_find #VaR cf
CVaRT_fr_find_DGM_CF[j]=-(mudg_find)-(desv_find)*dnorm(VaRstd_find)/(pnorm(VaRstd_find))
    
}
print("VaR forward find normal")
print(VaRNormal_find1)
print("CVaR forward find normal")
print(CVaRNormal_find1)
print("VaR forward find Delta Gamma Normal")
print(VaRT_fr_find_DGM)
print("CVaR forward find Delta Gamma Normal")
print(CVaRT_fr_find_DGM)
print("VaR forward find Delta Gamma CF")
print(VaRT_fr_find_DGM_CF)
print("VaR forward find Delta Gamma CF")
print(CVaRT_fr_find_DGM_CF)
#sum(V0_fwind)


[1] "VaR Normal total de Forward Indices"
          [,1]
[1,] -32718.65
[1] "VaR Normal total de Forward Indices"
          [,1]
[1,] -38846.78
[1] "VaR total de Forward Indices Delta Gamma Normal"
          [,1]
[1,] -32718.61
[1] "CVaR total de Forward Indices Delta Gamma Normal"
          [,1]
[1,] -38846.83
[1] "VaR total de Forward Indices Delta Gamma CF"
          [,1]
[1,] -32718.61
[1] "CVaR total de Forward Indices Delta Gamma CF"
          [,1]
[1,] -38846.83
[1] "VaR forward find normal"
          [,1] [,2]     [,3]
[1,] -31726.14    0 -383.918
[1] "CVaR forward find normal"
          [,1] [,2]      [,3]
[1,] -39785.86    0 -481.4486
[1] "VaR forward find Delta Gamma Normal"
          [,1] [,2]     [,3]
[1,] -31726.14    0 -383.918
[1] "CVaR forward find Delta Gamma Normal"
          [,1] [,2]      [,3]
[1,] -39785.86    0 -481.4487
[1] "VaR forward find Delta Gamma CF"
          [,1] [,2]      [,3]
[1,] -31726.14  NaN -383.8478
[1] "VaR forward find Delta Gamma CF"
        

Estos VaR y CVar, son un poco mas elevados a los pasados esto es porque se valuan de manera mas general y van teniendo mas riesgo si juntamos los indices 

## RIESGO OPCIONES

###Riesgo a nivel contrato y contrato-factor de riesgo


In [0]:
%%R
#Cálculo de matriz de pérdidas y ganancias opciones

#Cálculo de VaR y CVaR por contrato
#creación de matrices de VaR, VaR delta-normal,Var delta- gamma, VaR CF por contrato
DeltaX_oir=DeltaX_port_opc[,(1:n_if_opc[1])]  
m_oir=ncol(plazos_oir)    #número de factores de riesgo
m_opc=ncol(plazos_oir)

VaRmargcDN_oir=matrix(0,1,m_opc) #Var por contrato Delta Normal
VaRmargcDG_oir=matrix(0,1,m_opc) #Var por contrato Delta Gamma Normal
VaRmargcstd_oir=matrix(0,1,m_opc) #Var por contrato estandarizado 
VaRmargcCF_oir=matrix(0,1,m_opc) #Var por contrato Cornish-Fisher
CVaRmargcDN_oir=matrix(0,1,m_opc) #CVar por contrato Delta Normal
CVaRmargcDG_oir=matrix(0,1,m_opc) #CVar por contrato Delta Gamma Normal
CVaRmargcstd_oir=matrix(0,1,m_opc) #CVar por contrato estandarizado
CVaRmargcCF_oir=matrix(0,1,m_opc) #CVar por contrato Cornish-Fisher

##factores riesgo
#Spot
VaRmargcDN_fr_oir=matrix(0,3,m_opc) #Var por factor de riesgo Delta Normal
VaRmargcDG_fr_oir=matrix(0,3,m_opc) #Var por factor de riesgo Delta Gamma
VaRmargcstd_fr_oir=matrix(0,3,m_opc) #Var por factor de riesgo estandarizado 
VaRmargcCF_fr_oir=matrix(0,3,m_opc) #Var por factor de riesgo Cornish-Fisher
CVaRmargcDN_fr_oir=matrix(0,3,m_opc) #CVar por factor de riesgo Delta Normal
CVaRmargcDG_fr_oir=matrix(0,3,m_opc) #CVar por factor de riesgo Delta Gamma
CVaRmargcstd_fr_oir=matrix(0,3,m_opc) #Var por factor de riesgo estandarizado
CVaRmargcCF_fr_oir=matrix(0,3,m_opc) #CVar por factor de riesgo Cornish-Fisher
WFR_opc=diagv(c(1,1,1))


for (i in 1:(m_opc))
{
  delta_oir=t(delta_oir_T[seq(i,m_opc*3,by=m_opc)])  #me quedo con la fila 1,6,11; o 2,7,12 etc; agrupo las columnas para formar cada contrato
  tdelta_oir=t(delta_oir_T[seq(i,m_opc*3,by=m_opc)])*xp_oir[seq(i,m_opc*3,by=m_opc)]   #delta tilde
  gamma_oir=mgammaopc(x01[i],x02[i],K_oir[i],x03[i],plazos_oir[i],cp_oir[i],cs_oir,pr_oir,dct_oir,contratos_oir[i],nominal_oir)  #paso clave
  tgamma_oir=xp_oir[seq(i,m_opc*3,by=m_opc)]%*%t(xp_oir[seq(i,m_opc*3,by=m_opc)])*gamma_oir #gamma tilde
  Varmargcaux_oir=cov(DeltaX_oir[,seq(i,m_opc*3,by=m_opc)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_oir=(tgamma_oir%*%Varmargcaux_oir)
  tgammavar2_oir=(tgamma_oir%*%Varmargcaux_oir)%*%(tgamma_oir%*%Varmargcaux_oir)  
  mudg_oir= 0 #sum(diag(tgammavar_oir))/2+mu_oir[i] 
  desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

  mudn_oir=ifelse(mu_oir[i]>0,0,mu_oir[i])
  desvdn_oir=sqrt(tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

  VaRmargcDN_oir[i]=mudn_oir+desvdn_oir*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
  CVaRmargcDN_oir[i]=mudn_oir-desvdn_oir*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal

  VaRmargcDG_oir[i]=mudg_oir+desv_oir*qnorm(1-alpha) #VaR Delta Gamma Normal por contrato
  CVaRmargcDG_oir[i]=mudg_oir-desv_oir*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_oir=tgammavar2_oir%*%(tgamma_oir%*%Varmargcaux_oir)
  asimPmargc_oir=sum(diag(tgammavar3_oir))+3*tdelta_oir%*%Varmargcaux_oir%*%tgamma_oir%*%Varmargcaux_oir%*%t(tdelta_oir) #tercer momento
  asimPmargc_oir=asimPmargc_oir/(desv_oir)^3  #coef de asimetría
  VaRmargcstd_oir[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_oir  #VaR estandarizado
  CVaRmargcstd_oir[i] = 0
  VaRmargcCF_oir[i]=mudg_oir+desv_oir*VaRmargcstd_oir[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_oir[i]=-mudg_oir-desv_oir*(dnorm(VaRmargcstd_oir[i])/(pnorm(VaRmargcstd_oir[i])))

#Por factor de riesgo y por contrato
#descuento, spot y volatilidad
  for (j in 1:3) #para cada contrato iteramos 3 veces
  {
    WFRM_opc=matrix(0,3,3)  #PASO CLAVE
    WFRM_opc[j,j]=1  #PASO CLAVE
    delta_oir=t(delta_oir_T[seq(i,m_opc*3,by=m_opc)])*WFR_opc[j,]
    tdelta_oir=t(delta_oir_T[seq(i,m_opc*3,by=m_opc)])*xp_oir[seq(i,m_opc*3,by=m_opc)]*WFR_opc[j,]   #delta tilde
    gamma_oir=mgammaopc(x01[i],x02[i],K_oir[i],x03[i],plazos_oir[i],cp_oir[i],cs_oir,pr_oir,dct_oir,contratos_oir[i],nominal_oir)*WFRM_opc #paso clave
   #WFR_opc[j,] son varibles  guía, para el gradiente activamos las variables que son diferentes de cero.
   #print(WFRM_opc)
    tgamma_oir=xp_oir[seq(i,m_opc*3,by=m_opc)]%*%t(xp_oir[seq(i,m_opc*3,by=m_opc)])*gamma_oir #gamma tilde
    Varmargcaux_oir=cov(DeltaX_oir[,seq(i,m_opc*3,by=m_opc)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_oir=(tgamma_oir%*%Varmargcaux_oir)
    tgammavar2_oir=(tgamma_oir%*%Varmargcaux_oir)%*%(tgamma_oir%*%Varmargcaux_oir)  
    #mudg_oir=sum(diag(tgammavar_oir))/2+mu_oir[i]
    mudg_oir=0
    desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

    mudn_oir=0
    desvdn_oir=sqrt(tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

    VaRmargcDN_fr_oir[j,i]=mudn_oir+desvdn_oir*qnorm(1-alpha) #VaR por contrato-factor de riesgo Delta Normal
    CVaRmargcDN_fr_oir[j,i]=mudn_oir-desvdn_oir*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR por contrato-factor de riesgo Delta Normal

    VaRmargcDG_fr_oir[j,i]=mudg_oir+desv_oir*qnorm(1-alpha) #VaR Delta- Gamma por contrato-factor de riesgo
    CVaRmargcDG_fr_oir[j,i]=mudg_oir-desv_oir*(dnorm(qnorm(1-alpha))/(1-alpha)) #CVaR Delta- Gamma por contrato-factor de riesgo
    
    tgammavar3_oir=tgammavar2_oir%*%(tgamma_oir%*%Varmargcaux_oir)
    asimPmargc_oir=sum(diag(tgammavar3_oir))+3*tdelta_oir%*%Varmargcaux_oir%*%tgamma_oir%*%Varmargcaux_oir%*%t(tdelta_oir) #tercer momento
    asimPmargc_oir=asimPmargc_oir/(desv_oir)^3  #coef de asimetría
    VaRmargcstd_fr_oir[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_oir  #VaR estandarizado
    CVaRmargcstd_fr_oir = 0
    VaRmargcCF_fr_oir[j,i]=mudg_oir+desv_oir*VaRmargcstd_fr_oir[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_oir[j,i]=-mudg_oir-desv_oir*(dnorm(VaRmargcstd_fr_oir[j,i])/(pnorm(VaRmargcstd_fr_oir[j,i])))
  }


}

print("Valor a nivel contrato")
print(V0_oir)
print("")
print("VaR a nivel contrato Delta Normal")
print(VaRmargcDN_oir)
print("")
print("CVaR a nivel contrato Delta Normal")
print(CVaRmargcDN_oir)
print("")
print("VaR a nivel contrato Delta Gamma")
print(VaRmargcDG_oir)
print("")
print("CVaR a nivel contrato Delta Gamma")
print(CVaRmargcDG_oir)
print("")
print("VaR a nivel contrato CF")
print(VaRmargcCF_oir)
print("")
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_oir)

print("VaR a nivel contrato-factor de riesgo delta normal")
print(VaRmargcDN_fr_oir)
print("")
print("VaR a nivel contrato-factor de riesgo Delta Gamma Normal")
print(VaRmargcDG_fr_oir)
print("")
print("VaR a nivel contrato-factor de riesgo CF")
print(VaRmargcCF_fr_oir)
print("")
print("CVaR a nivel contrato-factor de riesgo delta normal")
print(CVaRmargcDN_fr_oir)
print("")
print("CVaR a nivel contrato-factor de riesgo Delta Gamma Normal")
print(CVaRmargcDG_fr_oir)
print("")
print("CVaR a nivel contrato-factor de riesgo CF")
print(CVaRmargcCF_fr_oir)


[1] "Valor a nivel contrato"
         [,1]     [,2]
[1,] 2.215311 0.121693
[1] ""
[1] "VaR a nivel contrato Delta Normal"
           [,1]       [,2]
[1,] -0.1046868 -0.0320249
[1] ""
[1] "CVaR a nivel contrato Delta Normal"
           [,1]        [,2]
[1,] -0.1312815 -0.04016052
[1] ""
[1] "VaR a nivel contrato Delta Gamma"
           [,1]        [,2]
[1,] -0.1046885 -0.03203328
[1] ""
[1] "CVaR a nivel contrato Delta Gamma"
           [,1]        [,2]
[1,] -0.1312837 -0.04017102
[1] ""
[1] "VaR a nivel contrato CF"
           [,1]        [,2]
[1,] -0.1045357 -0.03162541
[1] ""
[1] "CVaR a nivel contrato CF"
           [,1]        [,2]
[1,] -0.1311519 -0.03981979
[1] "VaR a nivel contrato-factor de riesgo delta normal"
            [,1]          [,2]
[1,] -0.01961880 -0.0003095892
[2,] -0.08530262 -0.0158174642
[3,] -0.04666979 -0.0290945145
[1] ""
[1] "VaR a nivel contrato-factor de riesgo Delta Gamma Normal"
            [,1]          [,2]
[1,] -0.01961894 -0.0003095894
[2,] -0.0853026

###Riesgo a nivel total y total factor de riesgo

In [0]:
%%R
VarDeltaX_oir=cov(DeltaX_oir)*(n/(n)) 

tdelta_oir= (deltaT_oir_T%*%VarDeltaX_oir)
mudn_oir=0
desvdn_oir=sqrt(tdelta_oir%*%VarDeltaX_oir%*%t(tdelta_oir))

VaRT_oir_DN=mudn_oir+desvdn_oir*qnorm(1-alpha)
CVaRT_oir_DN=mudn_oir-desvdn_oir*(dnorm(qnorm(1-alpha))/(1-alpha))

tgammavar_oir=(gammaT_oirT2%*%VarDeltaX_oir)
tgammavar2_oir=(tgammavar_oir)%*%(tgammavar_oir) 
mudg_oir= 0 #sum(diag(tgammavar_oir))/2+sum(mu_oir)
desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+deltaT_oir_T%*%VarDeltaX_oir%*%t(deltaT_oir_T))

VaRT_oir_DGM=(mudg_oir)+(desv_oir)*qnorm(1-alpha)  #VaR Gamma Normal
CVaRT_oir_DGM=(mudg_oir)-(desv_oir)*dnorm(qnorm(1-alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_oir=tgammavar2_oir%*%(tgammavar_oir)
asimP_oir=sum(diag(tgammavar3_oir))+3*deltaT_oir_T%*%VarDeltaX_oir%*%gammaT_oirT2%*%VarDeltaX_oir%*%t(deltaT_oir_T) #gamma tercer momento
asimP_oir=asimP_oir/(desv_oir)^3  #coef de asimetría
VaRstd_oir=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_oir  #VaR estandarizado
VaRT_oir_DGM_CF=(mudg_oir)+(desv_oir)*VaRstd_oir #VaR cf
CVaRT_oir_DGM_CF=(mudg_oir)-(desv_oir)*dnorm(VaRstd_oir)/(pnorm(VaRstd_oir))

print('VaR total Delta Normal')
print(VaRT_oir_DN)
print('CVaR total Delta Normal')
print(CVaRT_oir_DN)
print("")
print('VaR total Delta Gamma')
print(VaRT_oir_DGM)
print('CVaR total Delta Gamma')
print(CVaRT_oir_DGM)
print("")
print('VaR Cornish Fisher')
print(VaRT_oir_DGM_CF)
print('CVaR Cornish Fisher')
print(CVaRT_oir_DGM_CF)

VaRT_fr_oir_DN=matrix(0,1,3)
CVaRT_fr_oir_DN=matrix(0,1,3)
VaRT_fr_oir_DGM=matrix(0,1,3)
CVaRT_fr_oir_DGM=matrix(0,1,3)
VaRT_fr_oir_DGM_CF=matrix(0,1,3)
CVaRT_fr_oir_DGM_CF=matrix(0,1,3)

WFRT_opc=matrix(0,3,(m_opc*3))

#RIESGO TOTAL POR FACTOR DE RIESGO DE OPCIONES TASA DE INTERES

for (j in (1:3))
{
WFRT_opc[j,((j-1)*m_opc+1):(j*m_opc)]=matrix(1,1,m_opc)    #Matriz de diseño para 
WFRM_opc=matrix(0,(m_opc*3),(m_opc*3))  #PASO CLAVE
WFRM_opc[((j-1)*m_opc+1):(m_opc*j),((j-1)*m_opc+1):(m_opc*j)]=matrix(1,m_opc,m_opc)  #PASO CLAVE
gamma_oirtT2_fr=gammaT_oirT2*WFRM_opc #evaluación de gamma multiplicado por x^{2}
g_oirtT_fr=deltaT_oir_T*WFRT_opc[j,]
tgammavar_oir=(gamma_oirtT2_fr%*%VarDeltaX_oir)
tgammavar2_oir=(tgammavar_oir)%*%(tgammavar_oir) 
mudn_oir = 0
desvn_oir= sqrt(g_oirtT_fr%*%VarDeltaX_oir%*%t(g_oirtT_fr))
VaRT_fr_oir_DN[j]=(mudn_oir)+(desvn_oir)*qnorm(1-alpha) #VaR Delta Normal
CVaRT_fr_oir_DN[j]=(mudn_oir)-(desvn_oir)*dnorm(qnorm(1-alpha))/(1-alpha)

mudg_oir=0
desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+g_oirtT_fr%*%VarDeltaX_oir%*%t(g_oirtT_fr))
VaRT_fr_oir_DGM[j]=(mudg_oir)+(desv_oir)*qnorm(1-alpha) #VaR Gamma Normal
CVaRT_fr_oir_DGM[j]=(mudg_oir)-(desv_oir)*dnorm(qnorm(1-alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_oir=tgammavar2_oir%*%(tgammavar_oir)
asimP_oir=sum(diag(tgammavar3_oir))+3*g_oirtT_fr%*%VarDeltaX_oir%*%gamma_oirtT2_fr%*%VarDeltaX_oir%*%t(g_oirtT_fr) #gamma tercer momento
 #print(asimP_oir)
asimP_oir=asimP_oir/(desv_oir)^3  #coef de asimetría
 
VaRstd_oir=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_oir  #VaR estandarizado
#print(VaRstd_oir)
VaRT_fr_oir_DGM_CF[j]=(mudg_oir)+(desv_oir)*VaRstd_oir #VaR cf
CVaRT_fr_oir_DGM_CF[j]=(mudg_oir)-(desv_oir)*dnorm(VaRstd_oir)/(pnorm(VaRstd_oir))


}

print('VaR total por factor de riesgo Delta Normal')
print(VaRT_fr_oir_DN)
print('CVaR total por factor de riesgo Delta Normal')
print(CVaRT_fr_oir_DN)
print("")
print('VaR total por factor de riesgo Delta Gamma')
print(VaRT_fr_oir_DGM)
print('CVaR total por factor de riesgo Delta Gamma')
print(CVaRT_fr_oir_DGM)
print("")
print('VaR total por factor de riesgo Cornish Fisher')
print(VaRT_fr_oir_DGM_CF)
print('CVaR total por factor de riesgo Cornish Fisher')
print(CVaRT_fr_oir_DGM_CF)
print("")
print('Valoración de todas las opciones')
print(sum(V0_oir))


[1] "VaR total Delta Normal"
              [,1]
[1,] -0.0003457579
[1] "CVaR total Delta Normal"
              [,1]
[1,] -0.0004335943
[1] ""
[1] "VaR total Delta Gamma"
           [,1]
[1,] -0.1220141
[1] "CVaR total Delta Gamma"
           [,1]
[1,] -0.1530106
[1] ""
[1] "VaR Cornish Fisher"
           [,1]
[1,] -0.1216583
[1] "CVaR Cornish Fisher"
          [,1]
[1,] -0.152704
[1] "VaR total por factor de riesgo Delta Normal"
           [,1]        [,2]        [,3]
[1,] -0.0198301 -0.09007422 -0.06987064
[1] "CVaR total por factor de riesgo Delta Normal"
            [,1]       [,2]       [,3]
[1,] -0.02486774 -0.1129567 -0.0876206
[1] ""
[1] "VaR total por factor de riesgo Delta Gamma"
            [,1]        [,2]        [,3]
[1,] -0.01983024 -0.09007422 -0.06987905
[1] "CVaR total por factor de riesgo Delta Gamma"
            [,1]       [,2]        [,3]
[1,] -0.02486792 -0.1129567 -0.08763115
[1] ""
[1] "VaR total por factor de riesgo Cornish Fisher"
            [,1]        [,2]   

Observamos que para el VaR y CVaR a nivel total no se aleja tanto uno del otro, y los resultados de los 3 métodos son muy parecidos. Para los resultados por factor de riesgo observamos que la de tasa Spot se encuentra más alta que los otros dos factores y además el CVaR se aleja más proporcionalmente del VaR por factor de riego en los 3 métodos.

##RIESGO SWAPS

### Riesgo a nivel contrato y contrato-factor de riesgo

In [0]:
%%R
                      ### POR CONTRATO ###


#Valor de las derivadas evaluadas en el punto x0.
VaRcont=matrix(0,1,ncol(N))
CVaRcont=matrix(0,1,ncol(N))

V0f_dtt=dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_swc, Xvp[1,])
V0f_dtc=dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_swc, Xvp[1,])
V0f_dvp=dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_swc, Xvp[1,])
for (j in (1:ncol(N)))
{
  if(j==1)
  {
    mucont=sum(V0f_dtt[1:sum(N[1:j])])
    gcont=as.matrix(c(V0f_dtc[1:sum(N[1:j])],V0f_dvp[1:sum(N[1:j])]))
    gtcont=t(as.matrix(gcont*c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])])))
    VarCovXcont=cov(cbind(DeltaXtfwdt[,1:sum(N[1:j])],DeltaXvp[,1:sum(N[1:j])]))*(n/(n-1))	
    }
  else
  {
    mucont=sum(V0f_dtt[sum(N[1:j-1]+1):sum(N[1:j])])
    gcont=as.matrix(c(V0f_dtc[sum(N[1:j-1]+1):sum(N[1:j])],V0f_dvp[sum(N[1:j-1]+1):sum(N[1:j])]))
    gtcont=t(as.matrix(gcont*c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])])))
    VarCovXcont=cov(cbind(DeltaXtfwdt[,sum(N[1:j-1]+1):sum(N[1:j])],DeltaXvp[,sum(N[1:j-1]+1):sum(N[1:j])]))*(n/(n-1))
     }
  
  
    #VaR Delta Normal
  VaRcont[j]=sum(mucont)-sqrt(gtcont%*%VarCovXcont%*%t(gtcont))*qnorm(alpha)
  CVaRcont[j]=sum(mucont)-sqrt(gtcont%*%VarCovXcont%*%t(gtcont))*dnorm(qnorm(alpha))/(1-alpha)
}

print("VaR Normal por contrato")
print(VaRcont)
print("CVaR Normal por contrato")
print(CVaRcont)

                   ##Para obtener el VaR por contrato por factor de riesgo###


#Valor de las derivadas evaluadas en el punto x0.
alpha = .95
VaRcontfr_sw=matrix(0,2,ncol(N))
CVaRcontfr_sw=matrix(0,2,ncol(N))
WFRcont_sw <- rbind(c(1,0), c(0,1))

V0f_dtt_sw=dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_swc, Xvp[1,])
V0f_dtc_sw=dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_swc, Xvp[1,])
V0f_dvp_sw=dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_swc, Xvp[1,])
for (j in (1:ncol(N)))
{
    for(k in (1:2)){
  if(j==1)
  {
    mucont_sw=sum(V0f_dtt_sw[1:sum(N[1:j])])
    gcont_sw=as.matrix(c(V0f_dtc_sw[1:sum(N[1:j])],V0f_dvp_sw[1:sum(N[1:j])]))
    gtcont_sw=t(as.matrix(gcont_sw*c(XtfwdT_sw[1,1:sum(N[1:j])]*WFRcont_sw[k,1],Xvp_sw[1,1:sum(N[1:j])]*WFRcont_sw[k,2])))
    VarCovXcont_sw=cov(cbind(DeltaXtfwdt[,1:sum(N[1:j])],DeltaXvp[,1:sum(N[1:j])]))*(n/(n-1))	
  }
  else
  {
    mucont_sw=sum(V0f_dtt_sw[sum(N[1:j-1]+1):sum(N[1:j])])
    gcont_sw=as.matrix(c(V0f_dtc_sw[sum(N[1:j-1]+1):sum(N[1:j])],V0f_dvp_sw[sum(N[1:j-1]+1):sum(N[1:j])]))
    gtcont_sw=t(as.matrix(gcont_sw*c(XtfwdT_sw[1,sum(N[1:j-1]+1):sum(N[1:j])]*WFRcont_sw[k,1],Xvp_sw[1,sum(N[1:j-1]+1):sum(N[1:j])]*WFRcont_sw[k,2])))
    VarCovXcont_sw=cov(cbind(DeltaXtfwdt[,sum(N[1:j-1]+1):sum(N[1:j])],DeltaXvp[,sum(N[1:j-1]+1):sum(N[1:j])]))*(n/(n-1))
  }
  VaRcontfr_sw[k,j]=sum(mucont_sw)-sqrt(gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw))*qnorm(alpha)
  CVaRcontfr_sw[k,j]=sum(mucont_sw)-sqrt(gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw))*dnorm(qnorm(alpha))/(1-alpha)
}
}

print("VaR Normal por contrato por factor")
print(VaRcontfr_sw)
print("CVaR Normal por contrato por factor")
print(CVaRcontfr_sw)


          ##Para obtener el VaR por factor de riesgo##

#creación de vector de pesos por factor de riesgo.
WFR=rbind(c( XtfwdT[1,],Xvp[1,]*0),c( XtfwdT[1,]*0,Xvp[1,]))
VaRFr=matrix(0,1,2)
CVaRFr=matrix(0,1,2)

for (j in (1:2))
{
   
    g=cbind(dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_swc, Xvp[1,]),dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_swc, Xvp[1,]))
    
    #Multiplicación de la derivada por el punto x0
    gt=WFR[j,]*g
    VaRFr[j]=-sqrt(gt%*%VarCovX_sw%*%t(gt))*qnorm(alpha)
    CVaRFr[j]=-sqrt(gt%*%VarCovX_sw%*%t(gt))*dnorm(qnorm(alpha))/(1-alpha)
}
print("VaR Normal por factor")
print(VaRFr)
print("CVaR Normal por factor")
print(CVaRFr)


                      ### VaR CF y DELTA GAMMA NORMAL ###
#Cálculo de matriz de pérdidas y ganancias FUTUROS TDC

#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_sw=DeltaX_port_sw[,(1:n_if_sw[1])] 
m_sw=ncol(N)
m=ncol(N)
VaRmargc_sw=matrix(0,1,m_sw) #Var por contrato Delta Normal
VaRmargcstd_sw=matrix(0,1,m_sw) #Var por contrato estandarizado 
VaRmargcCF_sw=matrix(0,1,m_sw) #Var por contrato Cornish-Fisher
CVaRmargc_sw=matrix(0,1,m_sw) #CVar por contrato estandarizado
CVaRmargcCF_sw=matrix(0,1,m_sw) #CVar por contrato estandarizado

##factores riesgo
#tasa valor presente y cupon
VaRmargc_fr_sw=matrix(0,2,m_sw) #Var por contrato Delta Normal
VaRmargcstd_fr_sw=matrix(0,2,m_sw) #Var por contrato estandarizado 
VaRmargcCF_fr_sw=matrix(0,2,m_sw) #Var por contrato Cornish-Fisher
CVaRmargc_fr_sw=matrix(0,2,m_sw) #CVar por contrato estandarizado
CVaRmargcCF_fr_sw=matrix(0,2,m_sw) #CVar por contrato estandarizado
WFR=diagv(c(1,1))


for (i in 1:(m_sw))
{
  tdelta_sw=cbind(t(gt_sw[(sum(N[1:max(i-1,1)])*(i-1)+1):(sum(N[1:(i)]))]),t(gt_sw[(sum(N)+1+sum(N[1:max(i-1,1)])*(i-1)):(sum(N)+sum(N[1:(i)]))]))   #delta tilde
  tgamma_sw_c=matrix(0,N[i]*2,N[i]*2) 
  tgamma_sw_c[1:N[i],1:N[i]]=tgamma_sw[(sum(N[1:max(i-1,1)])*(i-1)+1):(sum(N[1:max(i-1,1)])*(i-1)+1):(sum(N[1:(i)]))]
  tgamma_sw_c[(N[i]+1):(N[i]*2),(N[i]+1):(N[i]*2)]=tgamma_sw[(sum(N)+1+sum(N[1:max(i-1,1)])*(i-1)):(sum(N)+sum(N[1:(i)])),(sum(N)+1+sum(N[1:max(i-1,1)])*(i-1)):(sum(N)+sum(N[1:(i)]))]
  
  Varmargcaux_sw=cov(cbind(DeltaX_sw[,(sum(N[1:max(i-1,1)])*(i-1)+1):(sum(N[1:(i)]))],DeltaX_sw[,(sum(N)+1+sum(N[1:max(i-1,1)])*(i-1)):(sum(N)+sum(N[1:(i)]))]))*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_sw=(tgamma_sw_c%*%Varmargcaux_sw)
  tgammavar2_sw=(tgamma_sw_c%*%Varmargcaux_sw)%*%(tgamma_sw_c%*%Varmargcaux_sw)  
  mudg_sw=sum(diag(tgammavar_sw))/2+sum(mu_sw[(sum(N[1:max(i-1,1)])*(i-1)+1):(sum(N[1:(i)]))])
  desv_sw=sqrt(sum(diag(tgammavar2_sw))/2+tdelta_sw%*%Varmargcaux_sw%*%t(tdelta_sw))

  VaRmargc_sw[i]=mudg_sw+desv_sw*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_sw[i]=-mudg_sw-desv_sw*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_sw=tgammavar2_sw%*%(tgamma_sw_c%*%Varmargcaux_sw)
  asimPmargc_sw=sum(diag(tgammavar3_sw))+3*tdelta_sw%*%Varmargcaux_sw%*%tgamma_sw_c%*%Varmargcaux_sw%*%t(tdelta_sw) #tercer momento
  asimPmargc_sw=asimPmargc_sw/(desv_sw)^3  #coef de asimetría
  VaRmargcstd_sw[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_sw  #VaR estandarizado
  VaRmargcCF_sw[i]=mudg_sw+desv_sw*VaRmargcstd_sw[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_sw[i]=-mudg_sw-desv_sw*(dnorm(VaRmargcstd_sw[i])/(pnorm(VaRmargcstd_sw[i])))

#Por factor de riesgo y por contrato
#spot, tasa extranjera, tasa nacional
  for (j in 1:2)
  {
      if(i==1){z=1}else{z=0}
   if(i==2){k=1}else{k=0}
    tdelta_sw=cbind(t(gt_sw[(sum(N[1:max(i-1,1)])*(i-1)+1):(sum(N[1:(i)]))])*z,t(gt_sw[(sum(N)+1+sum(N[1:max(i-1,1)])*(i-1)):(sum(N)+sum(N[1:(i)]))])*k)   #delta tilde
  tgamma_sw_c=matrix(0,N[i]*2,N[i]*2) 
  tgamma_sw_c[1:N[i],1:N[i]]=tgamma_sw[(sum(N[1:max(i-1,1)])*(i-1)+1):(sum(N[1:max(i-1,1)])*(i-1)+1):(sum(N[1:(i)]))]*z
  tgamma_sw_c[(N[i]+1):(N[i]*2),(N[i]+1):(N[i]*2)]=tgamma_sw[(sum(N)+1+sum(N[1:max(i-1,1)])*(i-1)):(sum(N)+sum(N[1:(i)])),(sum(N)+1+sum(N[1:max(i-1,1)])*(i-1)):(sum(N)+sum(N[1:(i)]))]*k
   
    #print(WFR[j,])
    Varmargcaux_sw=cov(cbind(DeltaX_sw[,(sum(N[1:max(i-1,1)])*(i-1)+1):(sum(N[1:(i)]))]*z,DeltaX_sw[,(sum(N)+1+sum(N[1:max(i-1,1)])*(i-1)):(sum(N)+sum(N[1:(i)]))]*k))*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_sw=(tgamma_sw_c%*%Varmargcaux_sw)
  tgammavar2_sw=(tgamma_sw_c%*%Varmargcaux_sw)%*%(tgamma_sw_c%*%Varmargcaux_sw)  
  #mudg_sw=sum(diag(tgammavar_sw))/2+sum(mu_sw[(sum(N[1:max(i-1,1)])*(i-1)+1):(sum(N[1:(i)]))])
  mudg_sw=0
  desv_sw=sqrt(sum(diag(tgammavar2_sw))/2+tdelta_sw%*%Varmargcaux_sw%*%t(tdelta_sw))

    VaRmargc_fr_sw[j,i]=mudg_sw+desv_sw*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_sw[j,i]=-mudg_sw-desv_sw*(dnorm(qnorm(1-alpha))/(1-alpha))

    tgammavar3_sw=tgammavar2_sw%*%(tgamma_sw_c%*%Varmargcaux_sw)
    asimPmargc_sw=sum(diag(tgammavar3_sw))+3*tdelta_sw%*%Varmargcaux_sw%*%tgamma_sw_c%*%Varmargcaux_sw%*%t(tdelta_sw) #tercer momento
    asimPmargc_sw=asimPmargc_sw/(desv_sw)^3  #coef de asimetría
    VaRmargcstd_fr_sw[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_sw  #VaR estandarizado
    VaRmargcCF_fr_sw[j,i]=mudg_sw+desv_sw*VaRmargcstd_fr_sw[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_sw[j,i]=-mudg_sw-desv_sw*(dnorm(VaRmargcstd_fr_sw[j,i])/(pnorm(VaRmargcstd_fr_sw[j,i])))
  }



}

print("VaR Delta-Gamma Normal por contrato ")
print(VaRmargc_sw)
print("CVaR Delta Normal por contrato ")
print(CVaRmargc_sw)
print("VaR Delta CF por contrato ")
print(VaRmargcCF_sw)
print("CVaR Delta CF por contrato ")
print(CVaRmargcCF_sw)
print("VaR por CF Por factor de riesgo y por contrato ")
print(VaRmargcCF_fr_sw)
print("CVaR por CF Por factor de riesgo y por contrato ")
print(CVaRmargcCF_fr_sw)
print("VaR Normal Por factor de riesgo y por contrato")
print(VaRmargc_fr_sw)
print("CVaR Normal Por factor de riesgo y por contrato")
print(CVaRmargc_fr_sw)


[1] "VaR Normal por contrato"
          [,1]       [,2]
[1,] -2.034095 -0.5971207
[1] "CVaR Normal por contrato"
          [,1]       [,2]
[1,] -2.547229 -0.7471846
[1] "VaR Normal por contrato por factor"
            [,1]         [,2]
[1,] -0.01420743 -0.006411568
[2,] -0.01420743 -0.006411568
[1] "CVaR Normal por contrato por factor"
            [,1]         [,2]
[1,] -0.01420743 -0.006411568
[2,] -0.01420743 -0.006411568
[1] "VaR Normal por factor"
          [,1]        [,2]
[1,] -2.496111 -0.04145992
[1] "CVaR Normal por factor"
          [,1]        [,2]
[1,] -3.130224 -0.05199241
[1] "VaR Delta-Gamma Normal por contrato "
     [,1] [,2]
[1,]   NA   NA
[1] "CVaR Delta Normal por contrato "
     [,1] [,2]
[1,]   NA   NA
[1] "VaR Delta CF por contrato "
     [,1] [,2]
[1,]   NA   NA
[1] "CVaR Delta CF por contrato "
     [,1] [,2]
[1,]   NA   NA
[1] "VaR por CF Por factor de riesgo y por contrato "
          [,1]         [,2]
[1,] -1.999764 -0.006854119
[2,] -1.999764 -0.006854119
[

De manera parecida a como nos salió en el método delta gamma normal y cornish fisher, notamos que la pata fija tiene mayor riesgo que la pata flotante. Sin embargo, solo es un valor de riesgo de dos pesos a pesar de tener un nominal de mucho más. Por lo tanto, podemos decir que está bien construida la valuación del swap, puesto que justamente el objetivo es que la tasa swap haga que a tiempo 0 valga cero el contrato. 

###Riesgo a nivel total y total factor de riesgo

In [0]:
%%R
VaRTotal_sw=sqrt(gt%*%VarCovX_sw%*%t(gt))*qnorm(alpha)#+dswap_dtt(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT,  plazocuponT, VTplazos, Xvp[1,])
CVaRTotal_sw=-sqrt(gt%*%VarCovX_sw%*%t(gt))*dnorm(qnorm(alpha))/(1-alpha)
print("VaR Normal Total Swaps")
print(VaRTotal_sw)
print("CVaR Normal Total Swaps")
print(CVaRTotal_sw)


                ### VaR POR CF Y DELTA GAMMA NORMAL##

#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO

VarDeltaX_sw=cov(DeltaX_sw)*(n/(n-1))
tgammavar_sw=(tgamma_sw%*%VarDeltaX_sw)
tgammavar2_sw=(tgammavar_sw)%*%(tgammavar_sw) 
mudg_sw=sum(diag(tgammavar_sw))/2+sum(mu_sw)
desv_sw=sqrt(sum(diag(tgammavar2_sw))/2+gt_sw%*%VarDeltaX_sw%*%t(gt_sw))

VaRT_sw_DGM=(mudg_sw)+(desv_sw)*qnorm(1-alpha)  #VaR Normal
CVaRT_sw_DGM=-(mudg_sw)-(desv_sw)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_sw=tgammavar2_sw%*%(tgammavar_sw)
asimP_sw=sum(diag(tgammavar3_sw))+3*gt_sw%*%VarDeltaX_sw%*%tgamma_sw%*%VarDeltaX_sw%*%t(gt_sw) #gamma tercer momento
asimP_sw=asimP_sw/(desv_sw)^3  #coef de asimetría
VaRstd_sw=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_sw  #VaR estandarizado
VaRT_sw_DGM_CF=(mudg_sw)+(desv_sw)*VaRstd_sw #VaR cf
CVaRT_sw_DGM_CF=-(mudg_sw)-(desv_sw)*dnorm(VaRstd_sw)/(pnorm(VaRstd_sw))

print("VaR Total de Swaps por Delta Gamma-Normal  ")
print(VaRT_sw_DGM)
print("CVaR Total de Swaps por Delta Gamma-Normal  ")
print(CVaRT_sw_DGM)
print("VaR Total de Swaps por Delta Gamma-CF  ")
print(VaRT_sw_DGM_CF)
print("CVaR Total de Swaps por Delta Gamma-CF  ")
print(CVaRT_sw_DGM_CF)

VaRT_fr_sw_DGM=matrix(0,1,2)
CVaRT_fr_sw_DGM=matrix(0,1,2)
VaRT_fr_sw_DGM_CF=matrix(0,1,2)
CVaRT_fr_sw_DGM_CF=matrix(0,1,2)

WFRT=matrix(0,sum(N)*2,sum(N)*2)
# WFRT[1,1:m]=matrix(1,1,m)
# WFRT[2,(m+1):(2*m)]=matrix(1,1,m)
# WFRT[3,(2*m+1):(3*m)]=matrix(1,1,m)

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:2))
{
WFRT[j,((j-1)*sum(N)+1):(j*sum(N))]=matrix(1,1,sum(N))    #Matriz de diseño para 
WFRM=matrix(0,(sum(N)*2),(sum(N)*2))  #PASO CLAVE
WFRM[((j-1)*sum(N)+1):(sum(N)*j),((j-1)*sum(N)+1):(sum(N)*j)]=matrix(1,sum(N),sum(N))  #PASO CLAVE    
t_gamma_sw_fr=tgamma_sw*WFRM #evaluación de gamma multiplicado por x^{2}
gt_sw_fr=gt_sw*WFRT[j,]
# print(WFRM)
# print(g_swtT_fr)
# print(gamma_swtT2_fr)
# print(WFRT)
tgammavar_sw=(t_gamma_sw_fr%*%VarDeltaX_sw)
tgammavar2_sw=(tgammavar_sw)%*%(tgammavar_sw) 
#mudg_sw=sum(diag(tgammavar_sw))/2+sum(mu_sw)
mudg_sw=0
desv_sw=sqrt(sum(diag(tgammavar2_sw))/2+gt_sw_fr%*%VarDeltaX_sw%*%t(gt_sw_fr))
VaRT_fr_sw_DGM[j]=(mudg_sw)+(desv_sw)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_sw_DGM[j]=-(mudg_sw)-(desv_sw)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_sw=tgammavar2_sw%*%(tgammavar_sw)
asimP_sw=sum(diag(tgammavar3_sw))+3*gt_sw_fr%*%VarDeltaX_sw%*%t_gamma_sw_fr%*%VarDeltaX_sw%*%t(gt_sw_fr) #gamma tercer momento
 #print(asimP_sw)
asimP_sw=asimP_sw/(desv_sw)^3  #coef de asimetría
 
VaRstd_sw=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_sw  #VaR estandarizado
#print(VaRstd_sw)
VaRT_fr_sw_DGM_CF[j]=(mudg_sw)+(desv_sw)*VaRstd_sw #VaR cf
CVaRT_fr_sw_DGM_CF[j]=-(mudg_sw)-(desv_sw)*dnorm(VaRstd_sw)/(pnorm(VaRstd_sw))

}


print("VaR Total de riesgo por Delta Gamma-Normal  ")
print(VaRT_fr_sw_DGM)
print("CVaR Total de riesgo por Delta Gamma-Normal  ")
print(CVaRT_fr_sw_DGM)
print("VaR Total de riesgo por Delta Gamma-CF  ")
print(VaRT_fr_sw_DGM_CF)
print("CVaR Total de riesgo por Delta Gamma-CF  ")
print(CVaRT_fr_sw_DGM_CF)



[1] "VaR Normal Total Swaps"
           [,1]
[1,] 0.04145992
[1] "CVaR Normal Total Swaps"
            [,1]
[1,] -0.05199241
[1] "VaR Total de Swaps por Delta Gamma-Normal  "
          [,1]
[1,] -2.490808
[1] "CVaR Total de Swaps por Delta Gamma-Normal  "
          [,1]
[1,] -3.077691
[1] "VaR Total de Swaps por Delta Gamma-CF  "
          [,1]
[1,] -2.490846
[1] "CVaR Total de Swaps por Delta Gamma-CF  "
          [,1]
[1,] -3.077724
[1] "VaR Total de riesgo por Delta Gamma-Normal  "
          [,1]        [,2]
[1,] -2.472303 -0.04514919
[1] "CVaR Total de riesgo por Delta Gamma-Normal  "
          [,1]       [,2]
[1,] -3.100367 -0.0566189
[1] "VaR Total de riesgo por Delta Gamma-CF  "
          [,1]        [,2]
[1,] -2.472303 -0.04512268
[1] "CVaR Total de riesgo por Delta Gamma-CF  "
          [,1]        [,2]
[1,] -3.100367 -0.05659606


Finalmente, podemos concluir que con este método se tiene más variabilidad en nuestro riesgo, lo que podría ser preocupante en activos financieros más riesgosos. No se trata de escoger el modelo que más nos convenga por lo que no podemos decidir cuál mide mejor nuestro riesgo con base a que nos dé menos variabilidad. Por lo tanto tomaríamos un punto medio para el valor en riesgo condicional para no sobrestimarlo ni estimarlo de menos.

#RIESGO INTEGRAL POR PORTAFOLIO

## RIESGO DE PORTAFOLIO DE BONOS, CETES Y BONDES

In [0]:
%%R

m_find=ncol(plazos_bdm)


#Cálculo de matriz de varianza covarianza de todos los factores de riesgo del PORTAFOLIO
VarDeltaX_port=as.matrix(cov(DeltaX_port)*(n)/(n-1))

mu_port=as.matrix(cbind(mu_bcc,mu_bm,mu_bd)) #derivada con respecto al tiempo
#INTEGRACIÓN DE PRIMERAS DERIVADAS (GRADIENTE)
gt_port=as.matrix(cbind(g_bcc,g_bm,g_bd)) #derivada con respecto las X's
#validar que ncol(X_port)=ncol(gt_port)


#INTEGRACIÓN DE MATRIZ HESSIANA

gammat_port=matrix(0,sum(n_if),sum(n_if))
gammat_port[1:n_if[1],1:n_if[1]]=gamma_bcct2
gammat_port[(sum(n_if[1:1])+1):(sum(n_if[1:2])),(sum(n_if[1:1])+1):(sum(n_if[1:2]))]=gamma_bmt2
gammat_port[(sum(n_if[1:2])+1):(sum(n_if[1:3])),(sum(n_if[1:2])+1):(sum(n_if[1:3]))]=gamma_bdt2

#print(gammat_port)
#print(head(DeltaX_s))

#CÁLCULO DE VAR Y CVAR A NIVEL PORTAFOLIO
tgammavar_port=(gammat_port%*%VarDeltaX_port)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
mudg_port=(sum(diag(tgammavar_port))/2+sum(mu_port))
desv_port=sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port))

#VaR Delta Normal
des_port_N = sqrt(gt_port%*%VarDeltaX_port%*%t(gt_port))
VaRT_port_DN = sum(mu_port) - des_port_N*qnorm(alpha)
CVaRT_port_DN= -sum(mu_port) - des_port_N*(dnorm(qnorm(alpha))/(1-alpha))

#VaR Deta Gamma Normal
VaRT_port_DGM=mudg_port+desv_port*qnorm(1-alpha) 
CVaRT_port_DGM=-mudg_port-desv_port*(dnorm(qnorm(1-alpha))/(1-alpha))

###VaR Cornish Fisher
tgammavar3_port=tgammavar2_port%*%(tgammavar_port)
asimP_port=sum(diag(tgammavar3_port))+3*gt_port%*%VarDeltaX_port%*%gammat_port%*%VarDeltaX_port%*%t(gt_port) #gamma tercer momento
asimP_port=asimP_port/(desv_port)^3  #coef de asimetría
VaRstd_port=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_port  #VaR estandarizado
VaRT_port_DGM_CF=mudg_port+desv_port*VaRstd_port #VaR cf
CVaRT_port_DGM_CF=-mudg_port-desv_port*(dnorm(VaRstd_port)/(pnorm(VaRstd_port)))

VaRT_port_DN/V0T_port
CVaRT_port_DN/V0T_port
VaRT_port_DGM/V0T_port
CVaRT_port_DGM/V0T_port
VaRT_port_DGM_CF/V0T_port
CVaRT_port_DGM_CF/V0T_port

print("VaR PORTAFOLIO DELTA NORMAL")
print(VaRT_port_DN)
print("VaR PORTAFOLIO DELTA GAMMA NORMAL")
print(VaRT_port_DGM)
print("VaR PORTAFOLIO CORNISH FISHER")
print(VaRT_port_DGM_CF)
print("CVAR PORTAFOLIO DELTA NORMAL")
print(CVaRT_port_DN)
print("CVAR PORTAFOLIO DELTA GAMMA NORMAL")
print(CVaRT_port_DGM)
print("CVaR PORTAFOLIO CORNISH FISHER")
print(CVaRT_port_DGM_CF)
print("VALUACIÓN PORTAFOLIO")
print(V0T_port)


#print(VaRT_port_DGM/V0T_port)
#print(CVaRT_port_DGM/V0T_port)
#print(VaRT_port_DGM_CF/V0T_port)
#print(CVaRT_port_DGM_CF/V0T_port)

#RIESGO TOTAL POR FACTOR DE RIESGO

V0_port=cbind(V0_bcc,V0_bm, V0_bdm)

#Se crea una matriz de diseño con ceros y unos indicando el factor de riesgo a querer obtener.
WFRT_port=matrix(0,3,ncol(DeltaX_port)) # Matriz para multiplicar al gradiente
WFRM_port1=matrix(0,ncol(DeltaX_port),ncol(DeltaX_port)) # Matriz para multiplicar a la hessiana
WFRM_port2=matrix(0,ncol(DeltaX_port),ncol(DeltaX_port)) # Matriz para multiplicar a la hessiana
WFRM_port3=matrix(0,ncol(DeltaX_port),ncol(DeltaX_port)) # Matriz para multiplicar a la hessiana

#CETES
a1=1 #columna inicial primer factor de riesgo cetes
b1=m_ftdc #columna final primer factor de riesgo
WFRT_port[1,a1:b1]=matrix(1,1,m_ftdc)
WFRM_port1[a1:b1,a1:b1]=matrix(1,m_ftdc,m_ftdc)

#BONdes
a2=m_find*3+1 #columna inicial segundo factor de riesgo
b2=a2-1+m_find #columna final segundo factor de riesgo
WFRT_port[2,a2:b2]=matrix(1,1,m_find)
WFRM_port2[a2:b2,a2:b2]=matrix(1,m_find,m_find)

# Bonos m
a31=m_bm+1 #columna inicial tercer factor de riesgo de tdc
b31=3*m_bm #columna final tercer factor de riesgo de tdc
WFRT_port[3,a31:b31]=matrix(1,1,m_bm*2)
WFRM_port3[a31:b31,a31:b31]=matrix(1,m_bm*2,m_bm*2)

WFRM_port=list(WFRM_port1,WFRM_port2,WFRM_port3)

VaRT_port_DN_fr=matrix(0,1,3)
CVaRT_port_DN_fr=matrix(0,1,3)
VaRT_port_DGM_fr=matrix(0,1,3)
CVaRT_port_DGM_fr=matrix(0,1,3)
VaRT_port_DGM_CF_fr=matrix(0,1,3)
CVaRT_port_DGM_CF_fr=matrix(0,1,3)

for (j in (1:3))
{
gt_port_fr=gt_port*WFRT_port[j,]
#mudg_port_fr=mu_port*WFRT_port[j,]
gammat_port_fr=gammat_port*WFRM_port[[j]] #evaluación de gamma multiplicado por x^{2}
#CÁLCULO DE VAR Y CVAR A NIVEL PORTAFOLIO
tgammavar_port=(gammat_port_fr%*%VarDeltaX_port)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
mudg_port=(sum(diag(tgammavar_port))/2+sum(mu_port))
#mudg_port=0
desv_port=sqrt(sum(diag(tgammavar2_port))/2+gt_port_fr%*%VarDeltaX_port%*%t(gt_port_fr))

#VaR Delta Normal
VaRT_port_DN_fr[j] = sum(mu_port) - sqrt(gt_port_fr%*%VarDeltaX_port%*%t(gt_port_fr))*qnorm(alpha) 
CVaRT_port_DN_fr[j] = sum(mu_port) - sqrt(gt_port_fr%*%VarDeltaX_port%*%t(gt_port_fr))*(dnorm(qnorm(alpha))/(1-alpha))

#VaR Delta Gamma Normal
VaRT_port_DGM_fr[j]=mudg_port+desv_port*qnorm(1-alpha) 
CVaRT_port_DGM_fr[j]=-mudg_port-desv_port*(dnorm(qnorm(1-alpha))/(1-alpha))

###VaR Cornish Fisher
tgammavar3_port=tgammavar2_port%*%(tgammavar_port)
asimP_port=sum(diag(tgammavar3_port))+3*gt_port_fr%*%VarDeltaX_port%*%gammat_port_fr%*%VarDeltaX_port%*%t(gt_port_fr) #gamma tercer momento
asimP_port=asimP_port/(desv_port)^3  #coef de asimetría
VaRstd_port=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_port  #VaR estandarizado
VaRT_port_DGM_CF_fr[j]=mudg_port+desv_port*VaRstd_port #VaR cf
CVaRT_port_DGM_CF_fr[j]=-mudg_port-desv_port*(dnorm(VaRstd_port)/(pnorm(VaRstd_port)))

}

print("VaR POR RIESGO DEL PORTAFOLIO DELTA NORMAL CETES, BONDES, BONO M")
print(VaRT_port_DN_fr)
print("VaR POR RIESGO DEL PORTAFOLIO DELTA GAMMA NORMAL CETES, BONDES, BONO M")
print(VaRT_port_DGM_fr)
print("VaR POR RIESGO DEL PORTAFOLIO CORNISH-FISHER")
print(VaRT_port_DGM_CF_fr)
print("CVaR POR RIESGO DEL PORTAFOLIO DELTA NORMAL")
print(CVaRT_port_DN_fr)
print("CVaR POR RIESGO DEL PORTAFOLIO DELTA GAMMA NORMAL")
print(CVaRT_port_DGM_fr)
print("CVaR POR RIESGO DEL PORTAFOLIO CORNISH-FISHER")
print(CVaRT_port_DGM_CF_fr)

[1] "VaR PORTAFOLIO DELTA NORMAL"
          [,1]
[1,] -6621.161
[1] "VaR PORTAFOLIO DELTA GAMMA NORMAL"
          [,1]
[1,] -6621.142
[1] "VaR PORTAFOLIO CORNISH FISHER"
          [,1]
[1,] -6621.111
[1] "CVAR PORTAFOLIO DELTA NORMAL"
          [,1]
[1,] -7188.205
[1] "CVAR PORTAFOLIO DELTA GAMMA NORMAL"
          [,1]
[1,] -7188.224
[1] "CVaR PORTAFOLIO CORNISH FISHER"
          [,1]
[1,] -7188.197
[1] "VALUACIÓN PORTAFOLIO"
[1] -865773
[1] "VaR POR RIESGO DEL PORTAFOLIO DELTA NORMAL CETES, BONDES, BONO M"
          [,1]     [,2]      [,3]
[1,] -573.3967 -673.718 -2184.556
[1] "VaR POR RIESGO DEL PORTAFOLIO DELTA GAMMA NORMAL CETES, BONDES, BONO M"
          [,1]     [,2]      [,3]
[1,] -573.3957 -673.718 -2184.559
[1] "VaR POR RIESGO DEL PORTAFOLIO CORNISH-FISHER"
          [,1]     [,2]      [,3]
[1,] -573.3939 -673.718 -2184.564
[1] "CVaR POR RIESGO DEL PORTAFOLIO DELTA NORMAL"
          [,1]      [,2]      [,3]
[1,] -593.3974 -719.2043 -2613.856
[1] "CVaR POR RIESGO DEL PORTAFOLIO

Aqui notamos que el portafolio en total tiene mas riesgo a cada riesgo 

##RIESGO DE PORTAFOLIO DE FORWARDS TDC E ÍNDICES

In [0]:
%%R

#Cálculo de matriz de varianza covarianza de todos los factores de riesgo del PORTAFOLIO
VarDeltaX_port_forwards=as.matrix(cov(DeltaX_port_forwards)*(n)/(n-1))

mu_port_forward=as.matrix(cbind(mu_ftdc,mu_find)) #derivada con respecto al tiempo
#INTEGRACIÓN DE PRIMERAS DERIVADAS (GRADIENTE)
gt_port_forward=as.matrix(cbind(g_ftdctT,g_findtT)) #derivada con respecto las X's
#validar que ncol(X_port_forwards)=ncol(gt_port_forward)


#INTEGRACIÓN DE MATRIZ HESSIANA

gammat_port=matrix(0,sum(n_if_forwards),sum(n_if_forwards))
gammat_port[1:n_if_forwards[1],1:n_if_forwards[1]]=gamma_ftdctT2
gammat_port[(sum(n_if_forwards[1])+1):(sum(n_if_forwards[1:2])),(sum(n_if_forwards[1])+1):(sum(n_if_forwards[1:2]))]=gamma_findtT2
#print(gammat_port)
#print(head(DeltaX_s))

#CÁLCULO DE VAR Y CVAR A NIVEL PORTAFOLIO
tgammavar_port=(gammat_port%*%VarDeltaX_port_forwards)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
mudg_port=(sum(diag(tgammavar_port))/2+sum(mu_port_forward))
desv_port=sqrt(sum(diag(tgammavar2_port))/2+gt_port_forward%*%VarDeltaX_port_forwards%*%t(gt_port_forward))
VaRT_port_DGM=mudg_port+desv_port*qnorm(1-alpha) #VaR Normal
CVaRT_port_DGM=-mudg_port-desv_port*(dnorm(qnorm(alpha))/(1-alpha))

#VaR Delta Normal
des_port_N_forwards = sqrt(gt_port_forward%*%VarDeltaX_port_forwards%*%t(gt_port_forward))
VaRT_port_DN_forwards = sum(mu_port_forward) - des_port_N_forwards*qnorm(alpha)
CVaRT_port_DN_forwards= -sum(mu_port_forward) - des_port_N_forwards*(dnorm(qnorm(alpha))/(1-alpha))


###VaR Cornish Fisher
tgammavar3_port=tgammavar2_port%*%(tgammavar_port)
asimP_port=sum(diag(tgammavar3_port))+3*gt_port_forward%*%VarDeltaX_port_forwards%*%gammat_port%*%VarDeltaX_port_forwards%*%t(gt_port_forward) #gamma tercer momento
asimP_port=asimP_port/(desv_port)^3  #coef de asimetría
VaRstd_port=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_port  #VaR estandarizado
VaRT_port_DGM_CF=mudg_port+desv_port*VaRstd_port #VaR cf
CVaRT_port_DGM_CF=-mudg_port-desv_port*(dnorm(VaRstd_port)/(pnorm(VaRstd_port)))

VaRT_port_DN_forwards/V0T_port_forwards
CVaRT_port_DN_forwards/V0T_port_forwards
VaRT_port_DGM/V0T_port_forwards
CVaRT_port_DGM/V0T_port_forwards
VaRT_port_DGM_CF/V0T_port_forwards
CVaRT_port_DGM_CF/V0T_port_forwards

print("VALOR DEL PORTAFOLIO")
print(V0T_port_forwards)
print("VaR portafolio por normal")
print(VaRT_port_DN_forwards)
print("CVaR portafolio por normal")
print(CVaRT_port_DN_forwards)
print("VaR portafolio por Delta Gamma Normal")
print(VaRT_port_DGM)
print("VaR portafolio por Delta Gamma CF")
print(VaRT_port_DGM_CF)
print("CVaR portafolio por Delta Gamma Normal")
print(CVaRT_port_DGM)
print("CVaR portafolio por Delta Gamma CF")
print(CVaRT_port_DGM_CF)



#print(VaRT_port_DGM/V0T_port_forwards)
#print(CVaRT_port_DGM/V0T_port_forwards)
#print(VaRT_port_DGM_CF/V0T_port_forwards)
#print(CVaRT_port_DGM_CF/V0T_port_forwards)


#RIESGO TOTAL POR FACTOR DE RIESGO

V0_port_forwards=cbind(V0_fwtdc, V0_fwind)

#Se crea una matriz de diseño con ceros y unos indicando el factor de riesgo a querer obtener.
WFRT_port=matrix(0,3,ncol(DeltaX_port_forwards)) # Matriz para multiplicar al gradiente
WFRM_port1=matrix(0,ncol(DeltaX_port_forwards),ncol(DeltaX_port_forwards)) # Matriz para multiplicar a la hessiana
WFRM_port2=matrix(0,ncol(DeltaX_port_forwards),ncol(DeltaX_port_forwards)) # Matriz para multiplicar a la hessiana
WFRM_port3=matrix(0,ncol(DeltaX_port_forwards),ncol(DeltaX_port_forwards)) # Matriz para multiplicar a la hessiana
#Tipo de cambio
#dado que sólo depende de los futuros de tipo de cambio sólo se hace de los primeros m_ftdc
a1=1 #columna inicial primer factor de riesgo
b1=m_ftdc #columna final primer factor de riesgo
WFRT_port[1,a1:b1]=matrix(1,1,m_ftdc)
WFRM_port1[a1:b1,a1:b1]=matrix(1,m_ftdc,m_ftdc)
#Acciones o índices
#dado que sólo depende de los futuros de índices sólo se hace de los primeros m_find pasados los 3*m_ftdc
a2=m_ftdc*3+1 #columna inicial segundo factor de riesgo
b2=a2-1+m_find #columna final segundo factor de riesgo
WFRT_port[2,a2:b2]=matrix(1,1,m_find)
WFRM_port2[a2:b2,a2:b2]=matrix(1,m_find,m_find)
#Tasa de interés
#La tasa de interés se usa en ambos forwards
#lo dividiremos en tasa de inters de tdc y de índices
#tdc
a31=m_ftdc+1 #columna inicial tercer factor de riesgo de tdc
b31=3*m_ftdc #columna final tercer factor de riesgo de tdc
WFRT_port[3,a31:b31]=matrix(1,1,m_ftdc*2)
WFRM_port3[a31:b31,a31:b31]=matrix(1,m_ftdc*2,m_ftdc*2)
#indice
a32=m_ftdc*3+m_find+1 #columna inicial tercer factor de riesgo de tdc
b32=ncol(DeltaX_port_forwards) #columna final tercer factor de riesgo de tdc
WFRT_port[3,a32:b32]=matrix(1,1,m_find*2)
WFRM_port3[a32:b32,a32:b32]=matrix(1,m_find*2,m_find*2)

WFRM_port=list(WFRM_port1,WFRM_port2,WFRM_port3)

# print(WFRT_port)
# print(WFRM_port)

VaRT_port_DN_fr=matrix(0,1,3)
CVaRT_port_DN_fr=matrix(0,1,3)
VaRT_port_DGM_fr=matrix(0,1,3)
CVaRT_port_DGM_fr=matrix(0,1,3)
VaRT_port_DGM_CF_fr=matrix(0,1,3)
CVaRT_port_DGM_CF_fr=matrix(0,1,3)

for (j in (1:3))
{
gt_port_fr=gt_port_forward*WFRT_port[j,]
#mudg_port_fr=mu_port_forward*WFRT_port[j,]
gammat_port_fr=gammat_port*WFRM_port[[j]] #evaluación de gamma multiplicado por x^{2}
#CÁLCULO DE VAR Y CVAR A NIVEL PORTAFOLIO
tgammavar_port=(gammat_port_fr%*%VarDeltaX_port_forwards)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
#mudg_port=(sum(diag(tgammavar_port))/2+sum(mu_port_forward))
mudg_port=0
desv_port=sqrt(sum(diag(tgammavar2_port))/2+gt_port_fr%*%VarDeltaX_port_forwards%*%t(gt_port_fr))
VaRT_port_DGM_fr[j]=mudg_port+desv_port*qnorm(1-alpha) #VaR Normal
CVaRT_port_DGM_fr[j]=-mudg_port-desv_port*(dnorm(qnorm(alpha))/(1-alpha))

#VaR Delta Normal
VaRT_port_DN_fr[j] = sum(mudg_port) - sqrt(gt_port_fr%*%VarDeltaX_port_forwards%*%t(gt_port_fr))*qnorm(alpha) 
CVaRT_port_DN_fr[j] = sum(mudg_port) - sqrt(gt_port_fr%*%VarDeltaX_port_forwards%*%t(gt_port_fr))*(dnorm(qnorm(alpha))/(1-alpha))


###VaR Cornish Fisher
tgammavar3_port=tgammavar2_port%*%(tgammavar_port)
asimP_port=sum(diag(tgammavar3_port))+3*gt_port_fr%*%VarDeltaX_port_forwards%*%gammat_port_fr%*%VarDeltaX_port_forwards%*%t(gt_port_fr) #gamma tercer momento
asimP_port=asimP_port/(desv_port)^3  #coef de asimetría
VaRstd_port=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_port  #VaR estandarizado
VaRT_port_DGM_CF_fr[j]=mudg_port+desv_port*VaRstd_port #VaR cf
CVaRT_port_DGM_CF_fr[j]=-mudg_port-desv_port*(dnorm(VaRstd_port)/(pnorm(VaRstd_port)))

}

print("VaR Por Riesgo por Normal")
print(VaRT_port_DN_fr)
print("CVaR Por Riesgo por Normal")
print(CVaRT_port_DN_fr)
print("VaR Por Riesgo por Delta Gamma Normal")
print(VaRT_port_DGM_fr)
print("VaR Por Riesgo por Delta Gamma CF")
print(VaRT_port_DGM_CF_fr)
print("CVaR Por Riesgo por Delta Gamma Normal")
print(CVaRT_port_DGM_fr)
print("CVaR Por Riesgo por Delta Gamma CF")
print(CVaRT_port_DGM_CF_fr)



[1] "VALOR DEL PORTAFOLIO"
[1] 4528345
[1] "VaR portafolio por normal"
          [,1]
[1,] -32714.43
[1] "CVaR portafolio por normal"
          [,1]
[1,] -38841.52
[1] "VaR portafolio por Delta Gamma Normal"
          [,1]
[1,] -32714.39
[1] "VaR portafolio por Delta Gamma CF"
          [,1]
[1,] -32714.39
[1] "CVaR portafolio por Delta Gamma Normal"
          [,1]
[1,] -38841.56
[1] "CVaR portafolio por Delta Gamma CF"
          [,1]
[1,] -38841.56
[1] "VaR Por Riesgo por Normal"
          [,1]      [,2]      [,3]
[1,] -18.12239 -31726.14 -383.9181
[1] "CVaR Por Riesgo por Normal"
         [,1]      [,2]      [,3]
[1,] -22.7262 -39785.86 -481.4488
[1] "VaR Por Riesgo por Delta Gamma Normal"
          [,1]      [,2]      [,3]
[1,] -18.12239 -31726.14 -383.9181
[1] "VaR Por Riesgo por Delta Gamma CF"
          [,1]      [,2]      [,3]
[1,] -18.12239 -31726.14 -383.8479
[1] "CVaR Por Riesgo por Delta Gamma Normal"
         [,1]      [,2]      [,3]
[1,] -22.7262 -39785.86 -481.4488
[1] "C

el riesgo que hay por portafolio no es tan diferente al que hay en el total de futuros tdc e indices esto se debe a que solo  se juntan ambos, ahora bien por factor es muchisimo mas pequeño yanda igual en el riesgo que se puede observar en los futuros de cada uno

# RIESGO INTEGRAL


In [0]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if_integral=matrix(0,8,1)
n_if_integral[1]=ncol(stock_prices_EQFX) #acciones y divisas
n_if_integral[2]=ncol(x_bcc) #cetes
n_if_integral[3]=ncol(X_bm) #bonos m
n_if_integral[4]=ncol(X_bd_ext) #bonde
n_if_integral[5]=ncol(X_ftdc) #fut tdc
n_if_integral[6]=ncol(X_find) #fut ind
n_if_integral[7]=ncol(X_sw) #swaps
n_if_integral[8]=ncol(X_oir) #opciones tasa de interés

#valor del portafolios

V0_port_integral=cbind(V0_bcc, V0_bm, V0_bdm, V0_fwtdc, V0_fwind, V0_sw, V0_oir)
V0T_port_integral=sum(V0_port_integral)
#print(V0T_port_integral)

#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port_integral=cbind(stock_prices_EQFX,x_bcc,X_bm,X_bd_ext,X_ftdc,X_find,X_sw,X_oir) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port_integral=as.matrix(log(X_port_integral[1:(n-1)]/X_port_integral[2:(n)]))
DeltaX_port_integral[is.nan(DeltaX_port_integral)] <- 0 #quitamos NaN
DeltaX_port_integral[is.na(DeltaX_port_integral)] <- 0 #quitamos Na
DeltaX_port_integral[is.infinite(DeltaX_port_integral)] <- 0 #quitamos Na


#Cálculo de matriz de varianza covarianza de todos los factores de riesgo del PORTAFOLIO
VarDeltaX_port_integral=as.matrix(cov(DeltaX_port_integral)*(n-1)/(n-2))


mu_port_integral=as.matrix(cbind(mu_acc_div,mu_bcc,mu_bm,mu_bd,mu_ftdc,mu_find,mu_sw,mu_oir)) #derivada con respecto al tiempo
#INTEGRACIÓN DE PRIMERAS DERIVADAS (GRADIENTE)
gt_port_integral=as.matrix(cbind(g_acc_divt,g_bcct,g_bmt,g_bdt,g_ftdctT,g_findtT,gt_sw,delta_oir_T)) #derivada con respecto las X's
#validar que ncol(X_port)=ncol(gt_port)


#INTEGRACIÓN DE MATRIZ HESSIANA

gammat_port_integral=matrix(0,sum(n_if_integral),sum(n_if_integral))
gammat_port_integral[1:n_if_integral[1],1:n_if_integral[1]]=gamma_acc_divt
gammat_port_integral[(sum(n_if_integral[1:1])+1):(sum(n_if_integral[1:2])),(sum(n_if_integral[1:1])+1):(sum(n_if_integral[1:2]))]=gamma_bcct2
gammat_port_integral[(sum(n_if_integral[1:2])+1):(sum(n_if_integral[1:3])),(sum(n_if_integral[1:2])+1):(sum(n_if_integral[1:3]))]=gamma_bmt2
gammat_port_integral[(sum(n_if_integral[1:3])+1):(sum(n_if_integral[1:4])),(sum(n_if_integral[1:3])+1):(sum(n_if_integral[1:4]))]=gamma_bdt2
gammat_port_integral[(sum(n_if_integral[1:4])+1):(sum(n_if_integral[1:5])),(sum(n_if_integral[1:4])+1):(sum(n_if_integral[1:5]))]=gamma_ftdctT2
gammat_port_integral[(sum(n_if_integral[1:5])+1):(sum(n_if_integral[1:6])),(sum(n_if_integral[1:5])+1):(sum(n_if_integral[1:6]))]=gamma_findtT2
gammat_port_integral[(sum(n_if_integral[1:6])+1):(sum(n_if_integral[1:7])),(sum(n_if_integral[1:6])+1):(sum(n_if_integral[1:7]))]=tgamma_sw
gammat_port_integral[(sum(n_if_integral[1:7])+1):(sum(n_if_integral[1:8])),(sum(n_if_integral[1:7])+1):(sum(n_if_integral[1:8]))]=gammaT_oirT2



#CÁLCULO DE VAR Y CVAR A NIVEL PORTAFOLIO
tgammavar_port_integral=(gammat_port_integral%*%VarDeltaX_port_integral)
tgammavar2_port_integral=(tgammavar_port_integral)%*%(tgammavar_port_integral) 
mu_portT_integral=ifelse((sum(diag(tgammavar_port_integral))/2+sum(mu_port_integral))>0,0,(sum(diag(tgammavar_port_integral))/2+sum(mu_port_integral)))
desv_portT_integral=(sqrt(sum(diag(tgammavar2_port_integral))/2+gt_port_integral%*%VarDeltaX_port_integral%*%t(gt_port_integral)))

#VaR delta gamma normal
VaRT_port_DGM_integral=mu_portT_integral+desv_portT_integral*qnorm(1-alpha) #VaR Normal
CVaRT_port_DGM_integral=mu_portT_integral-desv_portT_integral*dnorm(qnorm(alpha))/(1-alpha)

#VaR Normal
VaRT_port_DN_integral=sum(mu_port_integral)+sqrt(gt_port_integral%*%VarDeltaX_port_integral%*%t(gt_port_integral))*qnorm(1-alpha) #VaR Normal
CVaRT_port_DN_integral=-sum(mu_port_integral)-sqrt(gt_port_integral%*%VarDeltaX_port_integral%*%t(gt_port_integral))*dnorm(qnorm(alpha))/(1-alpha)


tgammavar2_port_integral[(sum(n_if_integral[1:5])+1):(sum(n_if_integral[1:6])),(sum(n_if_integral[1:5])+1):(sum(n_if_integral[1:6]))]

###VaR Cornish Fisher
tgammavar3_port_integral=tgammavar2_port_integral%*%(tgammavar_port_integral)
asimP_port_integral=sum(diag(tgammavar3_port_integral))+3*gt_port_integral%*%VarDeltaX_port_integral%*%gammat_port_integral%*%VarDeltaX_port_integral%*%t(gt_port_integral) #gamma tercer momento
asimP_port_integral=asimP_port_integral/(sqrt(sum(diag(tgammavar2_port_integral))/2+gt_port_integral%*%VarDeltaX_port_integral%*%t(gt_port_integral)))^3  #coef de asimetría
VaRstd_port_integral=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_port_integral  #VaR estandarizado
VaRT_port_DGM_CF_integral=mu_portT_integral+desv_portT_integral*VaRstd_port_integral #VaR cf
CVaRT_port_DGM_CF_integral=mu_portT_integral-desv_portT_integral*dnorm(VaRstd_port_integral)/(pnorm(VaRstd_port_integral))

print("VaR Normal total ")
print(VaRT_port_DN_integral/V0T_port_integral)
print("CVaR Normal total" )
print(CVaRT_port_DN_integral/V0T_port_integral)
print("VaR Delta Gamma Normal total ")
print(VaRT_port_DGM_integral/V0T_port_integral)
print("CVaR Delta Gamma Normal total ")
print(CVaRT_port_DGM_integral/V0T_port_integral)
print("VaR Delta Gamma CF total ")
print(VaRT_port_DGM_CF_integral/V0T_port_integral)
print("CVaR Delta Gamma CF total ")
print(CVaRT_port_DGM_CF_integral/V0T_port_integral)


[1] "VaR Normal total "
             2010-03-30
2010-03-30 -0.009832062
[1] "CVaR Normal total"
            2010-03-30
2010-03-30 -0.01143556
[1] "VaR Delta Gamma Normal total "
             2010-03-30
2010-03-30 -0.009832045
[1] "CVaR Delta Gamma Normal total "
           2010-03-30
2010-03-30  -0.012229
[1] "VaR Delta Gamma CF total "
             2010-03-30
2010-03-30 -0.009832045
[1] "CVaR Delta Gamma CF total "
           2010-03-30
2010-03-30  -0.012229
